In [ ]:
%matplotlib inline
import random
import glob
import os
from PIL import Image
import torch
import torchvision
from torch.autograd import Variable
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from util.util import load_opt

def show(img, show=True):
    npimg = img.numpy()
    npimg = ((npimg / 2.0) + 0.5) * 255.0
    npimg = npimg.astype(np.uint8)
    if show:
        plt.figure(figsize = (20,20))
        plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    return Image.fromarray(npimg.transpose((1,2,0)))

import util.util as util
from models import networks
from options.test_options import TestOptions

def load_network_with_path(network, network_label, epoch_label='latest', path='./checkpoints/danbooru_celeba_cyclegan/'):
    save_filename = '%s_net_%s.pth' % (epoch_label, network_label)
    save_path = os.path.join(path, save_filename)
    network.load_state_dict(torch.load(save_path))

def get_trans(crop=None, size=None):
    transform_list_show = []
    transform_list = []
    if crop is not None:
        opt.loadSize = crop
    if size is not None:
        opt.fineSize = size
    osize = [opt.loadSize, opt.loadSize]
    transform_list_show += [transforms.Scale(osize, Image.BICUBIC),
                       transforms.RandomCrop(opt.fineSize),
                       transforms.ToTensor()]
    img_show = transforms.Compose(transform_list_show)
    transform_list = transform_list_show + [transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]
    img_pre = transforms.Compose(transform_list)
    return img_show, img_pre    

def get_idtrans(crop=None, size=None):
    transform_list_show = []
    transform_list = []
    transform_list_show += [transforms.ToTensor()]
    img_show = transforms.Compose(transform_list_show)
    transform_list = transform_list_show + [transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]
    img_pre = transforms.Compose(transform_list)
    return img_show, img_pre   

def to_input(img, trans, size):
    img = trans(img)
    input_ = img.view(-1, 3, size, size)
    real = Variable(input_.cuda())
    return real

def test_img(img, net, trans, size=128, eval_mode=True, bn_eval=False, drop_eval=True):
    img = trans(img)
    input_ = img.unsqueeze(0)
    real = Variable(input_.cuda())
    set_eval(net, bn_eval, drop_eval)
    fake = net.forward(real)
    return fake

def get_stack_nets(path, which_epoch = 'latest'):
    gpu_ids = [0]
    Tensor = torch.cuda.FloatTensor
    opt = load_opt(path)
    netG_A = networks.define_G(opt.input_nc, opt.output_nc,
                            opt.ngf, opt.which_model_netG, opt.norm, False, opt.init_type, 
                            gpu_ids, n_upsampling=3, opt=opt)
    netG_B = networks.define_G(opt.output_nc, opt.input_nc,
                            opt.ngf, opt.which_model_netG, opt.norm, False, opt.init_type, 
                            gpu_ids, n_upsampling=3, opt=opt)
    netG_A_pre = networks.define_G(opt.input_nc, opt.output_nc,
                            opt.ngf, opt.which_model_netG, opt.norm, False, opt.init_type, 
                            gpu_ids, n_downsampling=3, opt=opt)
    netG_B_pre = networks.define_G(opt.output_nc, opt.input_nc,
                            opt.ngf, opt.which_model_netG, opt.norm, False, opt.init_type, 
                            gpu_ids, n_downsampling=3, opt=opt)

    load_network_with_path(netG_A,  'G_A', which_epoch, path)
    load_network_with_path(netG_B,  'G_B', which_epoch, path)
    load_network_with_path(netG_A_pre,  'G_A_pre', which_epoch, path)
    load_network_with_path(netG_B_pre,  'G_B_pre', which_epoch, path)
        
    netG_A.cuda()
    netG_B.cuda()
    netG_A_pre.cuda()
    netG_B_pre.cuda()
    
    return {'A':netG_A, 'B':netG_B, 'A_pre':netG_A_pre, 'B_pre':netG_B_pre}

def get_stack_nets(path, which_epoch = 'latest', skip=True):
    gpu_ids = [0]
    Tensor = torch.cuda.FloatTensor
    opt = load_opt(path)
    opt.max_ngf = 256
    netG_A = networks.define_G(opt.input_nc if not skip else opt.input_nc * 2, opt.output_nc,
                            opt.ngf, opt.which_model_netG, opt.norm, not opt.no_dropout, opt.init_type, 
                            gpu_ids, n_upsampling=opt.n_upsample, opt=opt)
    netG_B = networks.define_G(opt.input_nc if not skip else opt.input_nc * 2, opt.input_nc,
                            opt.ngf, opt.which_model_netG, opt.norm, not opt.no_dropout, opt.init_type, 
                            gpu_ids, n_upsampling=opt.n_upsample, opt=opt)
    if 'pre_path' in vars(opt).keys():
        opt_pre = load_opt(opt.pre_path)
        netG_A_pre = networks.define_G(opt.input_nc, opt.output_nc,
                                opt_pre.ngf, opt_pre.which_model_netG, opt_pre.norm, not opt_pre.no_dropout, opt_pre.init_type, 
                                gpu_ids, n_downsampling=3, opt=opt_pre)
        netG_B_pre = networks.define_G(opt.output_nc, opt.input_nc,
                                opt_pre.ngf, opt.which_model_netG, opt_pre.norm, not opt_pre.no_dropout, opt_pre.init_type, 
                                gpu_ids, n_downsampling=3, opt=opt_pre)
    else:
        opt.pre_path = ''
        netG_A_pre = networks.define_G(opt.input_nc, opt.output_nc,
                            opt.ngf, opt.which_model_netG, opt.norm, False, opt.init_type, 
                            gpu_ids, n_downsampling=3, opt=opt)
        netG_B_pre = networks.define_G(opt.output_nc, opt.input_nc,
                            opt.ngf, opt.which_model_netG, opt.norm, False, opt.init_type, 
                            gpu_ids, n_downsampling=3, opt=opt)

    load_network_with_path(netG_A,  'G_A', which_epoch, path)
    load_network_with_path(netG_B,  'G_B', which_epoch, path)
    load_network_with_path(netG_A_pre,  'G_A_pre', which_epoch, path if opt.pre_path == '' else opt.pre_path)
    load_network_with_path(netG_B_pre,  'G_B_pre', which_epoch, path if opt.pre_path == '' else opt.pre_path)
        
    netG_A.cuda()
    netG_B.cuda()
    netG_A_pre.cuda()
    netG_B_pre.cuda()
    
    return {'A':netG_A, 'B':netG_B, 'A_pre':netG_A_pre, 'B_pre':netG_B_pre}

def get_nets(path, which_epoch = 'latest'):
    gpu_ids = [0]
    Tensor = torch.cuda.FloatTensor
    opt = load_opt(path)
    opt.not_caffe = True
    if 'caffe' in path or 'top' in path:
        opt.not_caffe = False
    netG_A = networks.define_G(opt.input_nc, opt.output_nc,
                                  opt.ngf, opt.which_model_netG,
                                  opt.norm,  not opt.no_dropout, opt.init_type,
                                  gpu_ids, opt=opt)
    load_network_with_path(netG_A, 'G_A', which_epoch, path)
    
    netG_B = networks.define_G(opt.input_nc, opt.output_nc,
                                  opt.ngf, opt.which_model_netG,
                                  opt.norm,  not opt.no_dropout, opt.init_type,
                                  gpu_ids, opt=opt)
    load_network_with_path(netG_B, 'G_B', which_epoch, path)
    
    netG_A.cuda()
    netG_B.cuda()
    
    return {'A':netG_A, 'B':netG_B}

def get_nets_dis(path, which_epoch = 'latest'):
    gpu_ids = [0]
    Tensor = torch.cuda.FloatTensor
    opt = load_opt(path)
    opt.caffe=False
    netD_A = networks.define_D(opt.output_nc, opt.ndf,
                                    opt.which_model_netD,
                                    opt.n_layers_D, opt.norm, False, opt.init_type, gpu_ids, opt=opt)
    netD_B = networks.define_D(opt.input_nc, opt.ndf,
                                    opt.which_model_netD,
                                    opt.n_layers_D, opt.norm, False, opt.init_type, gpu_ids, opt=opt)
    load_network_with_path(netD_A, 'D_A', which_epoch, path)
    load_network_with_path(netD_B, 'D_B', which_epoch, path)
    
    netD_A.cuda()
    netD_B.cuda()
    
    return {'A':netD_A, 'B':netD_B}

opt = TestOptions().parse()
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

In [ ]:
net_0['B']

In [ ]:
opt.dataroot = './datasets/getchu_aisacelebaselfie_aligned_v6/'
phase = 'train'
B_path = glob.glob(os.path.join(opt.dataroot,phase+'B/*'))
B_mask_path = glob.glob(os.path.join('./datasets/getchu_man_v1_masked_blur/trainB/*'))
A_path = glob.glob(os.path.join(opt.dataroot,phase+'A/*'))
# AB_path = glob.glob(os.path.join(opt.dataroot,phase+'/*'))

img_show, img_pre = get_trans()
img_show_64, img_pre_64 = get_trans(64, 64)
img_show_128, img_pre_128 = get_trans(128, 128)
img_show_256, img_pre_256 = get_trans(256, 256)


In [ ]:
# net_1 = get_nets('./tmp/trans/Get_Cel_Ali_Cln_128_d4_0/')
# #net_1_d = get_nets_dis('./checkpoints/getchu_celeba_aligned_128/', which_epoch='40')
# net_1 = get_nets('./checkpoints/Get_AisCelSel_128_true4_masked_blur_0/')
# net_2 = get_nets('./checkpoints/Get_AisCelSel_128_true4_masked_blur_1/')
net_0 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d3_aff_gp10_caffe_0/')
net_1 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d3_aff_gp10_caffe_0/bk_0122/')
net_2 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d3_aff_gp10_caffe_0/bk_0123/')
net_3 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d3_aff_gp10_caffe_1/')
net_4 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d3_aff_gp2p5_caffe_0/')
net_5 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d3_aff_gp2p5_caffe_1/')
net_6 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0/')
net_7 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0/bk_0118/')
net_8 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0/bk_0122/')
net_9 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0/bk_0123/')
net_10 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1/')
net_11 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1/bk_0118/')
net_12 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1/bk_0122/')
net_13 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_trued4_aff_gp10_caffe_0/')
net_14 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_trued4_aff_gp10_caffe_1/')
net_15 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_trued4_aff_gp10_caffe_1/bk_0123/')
net_16 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_trued4_aff_gp10_caffe_1/bk_0123_v2/')
net_17 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d3_aff_gp2p5_caffe_3/')
net_18 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_d3_aff_gp2p5_caffe_3/bk_0123/')
net_19 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_trued4_aff_gp10_caffe_3/')
net_20 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_trued4_aff_gp10_caffe_3/bk_0123/')
net_21 = get_nets('/data2/minjunli/outbox/Get_AisCelSelf_128_trued4_aff_gp10_caffe_3/bk_0123_v2/')
net_22 = get_nets('./checkpoints/Get_AisCel_128_v6_d4_caffe_aff_0/')
net_23 = get_nets('./checkpoints/Get_AisCel_128_v6_d4_caffe_aff_1/')
net_24 = get_nets('./checkpoints/Get_AisCel_128_v6_d4_caffe_aff_masked_0/')
net_25 = get_nets('./checkpoints/Get_AisCel_128_v6_d4_caffe_aff_masked_1/')
net_26 = get_nets('./checkpoints/load_Get_AisCel_128_4_aff_0/')
net_27 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0_cont/')
net_28 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0_v6/')
net_29 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0_v7/')
net_30 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_cont/')
net_31 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_v6/')
net_32 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_v7/')
#net_33 = get_nets('./checkpoints/load_Get_AisCelSelf_128_trued4_aff_gp10_caffe_0/')
net_33 = get_nets('/data2/minjunli/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0/')
net_34 = get_nets('/data2/minjunli/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1/')
net_35 = get_nets('./checkpoints/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0/')
net_36 = get_nets('./checkpoints/Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1/')
#net_37 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_0_v6_color/')
net_37 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_masked_idt_0/')
net_38 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_masked_idt_1/')
net_39 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_masked_idt_2/')
net_40 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_masked_idt_3/')
net_41 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_masked_ft/')
net_42 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_masked_ft_1/')
net_43 = get_nets('./checkpoints/load_Get_AisCelSelf_128_d4_aff_gp2p5_caffe_1_masked_ft_regp/')
net_44 = get_nets('./checkpoints/load_top1_idt_ft/')
net_45 = get_nets('./checkpoints/load_top1_lr_ft/')
net_46 = get_nets('./checkpoints/load_top2_idt_ft/')
net_47 = get_nets('./checkpoints/load_top2_ft_ft/')
# for i in range(1,11):
#     exec('net_47_%d = get_nets("./checkpoints/load_top2_ft_ft/", which_epoch="%d")'%(i,i))
net_48 = get_nets('./checkpoints/load_top4_idt_ft/')
net_49 = get_nets('./checkpoints/load_top4_lr_ft/')
# for i in range(1,11):
#     exec('net_49_%d = get_nets("./checkpoints/load_top4_lr_ft/", which_epoch="%d")'%(i,i))
net_50 = get_nets('./checkpoints/load_top5_idt_ft/')
net_51 = get_nets('./checkpoints/load_top5_lr_ft/')
net_52 = get_nets('./checkpoints/load_net1_lr_ft/')
# for i in range(1,11):
#     exec('net_51_%d = get_nets("./checkpoints/load_top5_lr_ft/", which_epoch="%d")'%(i,i))


# net_2 = get_nets('../trans/Get_AisCel_128_d4_gpreweight_1/')
# net_1 = get_nets('./checkpoints/load_Get_AisCel_128_4_aff_0/')
# #net_1_d = get_nets_dis('./checkpoints/getchu_celeba_aligned_128/', which_epoch='40')
# net_2 = get_nets('./checkpoints/load_Get_AisCel_128_4_aff_1/')
# net_3 = get_nets('./checkpoints/Get_AisCel_128_4_masked_blur_0/')
# net_4 = get_nets('./checkpoints/Get_AisCel_128_4_masked_blur_1/')
# net_5 = get_nets('./tmp/trans/Get_AisCel_128_4_0_0//')
# net_6 = get_nets('./tmp/trans/Get_AisCel_128_d4_gpreweight_1//')
# net_7 = get_nets('./checkpoints/Get_AisCel_128_d4_masked_blur_aff_0/')
# net_8 = get_nets('./checkpoints/Get_AisCel_128_d4_masked_blur_aff_1/')
# net_9 = get_nets('./checkpoints/Get_AisCel_128_trued4_masked_blur_0/')
# net_10 = get_nets('./checkpoints/Get_AisCel_128_trued4_masked_blur_1/')
# net_11 = get_nets('./checkpoints/Get_Men_128_aff_0/')
# net_12 = get_nets('./checkpoints/Get_Men_128_aff_1/')
# net_13 = get_nets('./checkpoints/Get_Men_128_masked_blur_0/')
# net_14 = get_nets('./checkpoints/Get_Men_128_masked_blur_1/')

# net_11 = get_nets('./checkpoints/Get_AisCelSel_128_true4_masked_blur_0/')
# net_12 = get_nets('./checkpoints/Get_AisCelSel_128_true4_masked_blur_1/')

In [ ]:
net_47 = get_nets('./checkpoints/load_top2_ft_ft/')
for i in range(1,11):
    exec('net_47_%d = get_nets("./checkpoints/load_top2_ft_ft/", which_epoch="%d")'%(i,i))
net_48 = get_nets('./checkpoints/load_top4_idt_ft/')
net_49 = get_nets('./checkpoints/load_top4_lr_ft/')
for i in range(1,11):
    exec('net_49_%d = get_nets("./checkpoints/load_top4_lr_ft/", which_epoch="%d")'%(i,i))
net_50 = get_nets('./checkpoints/load_top5_idt_ft/')
net_51 = get_nets('./checkpoints/load_top5_lr_ft/')
for i in range(1,11):
    exec('net_51_%d = get_nets("./checkpoints/load_top5_lr_ft/", which_epoch="%d")'%(i,i))

In [ ]:
import argparse
import base64
import requests
import cStringIO
from io import BytesIO
import json
from PIL import Image

URL = 'http://100.102.36.11:30001'


def _to_img(img, image_width, image_height):
    i = img
    if image_width != None and image_height != None:
        i = i.resize((image_width, image_height))
    buf = cStringIO.StringIO()
    i.save(buf, format="JPEG")
    buf = base64.encodestring(buf.getvalue())
    buf += "=" * (-len(buf) % 4)
    q = buf
    # print(q)
    return q

def _get_img(img_base64, image_width, image_height):
    '''base64 to numpy'''
    #print(img_base64)
    i = Image.open(BytesIO(base64.decodestring(img_base64))).convert('RGB')
    #print(i.size)
    imw, imh = i.size
    if image_width != None and image_height != None:
        i = i.resize((image_width, image_height))
    return i



In [ ]:

B_p = random.choice(B_path)
name = B_p.split('/')[-1]

img = Image.open(B_p).convert('RGB').resize((128,128), Image.LANCZOS)#.crop((2,2,126,126))


image_width =  None
image_height = None

image_base64 = _to_img(img, image_width, image_height)

r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})
print r.status_code
#print r.content
js = r.json()
#print js
mask = _get_img(js['prob'], image_width, image_height)
#image_matting = _get_img(js['img_data'], image_width, image_height)

img_npy = np.asarray(img)
mask = mask.point(lambda p: p > 50 and 255) 
#mask = mask.point(lambda p: p < 128 or 0)

mask_npy = np.asarray(mask) / 255

mask_inv_npy = 1 - mask_npy
white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255


img_masked_npy = np.multiply(img_npy, mask_npy) + np.multiply(white_img_npy, mask_inv_npy)
img_masked = Image.fromarray(img_masked_npy)

show(torchvision.utils.make_grid([img_pre_128(img), img_pre_128(mask), img_pre_128(img_masked)]), show=False)




In [ ]:
#test align
import numpy as np
import subprocess
import os
from IPython.html.widgets import interact
from PIL import Image
import random
from PIL import ImageOps

def angle_between_2_points(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    tan = (y2 - y1) / (x2 - x1)
    return np.degrees(np.arctan(tan))

def align_eye_pad_ailab(im, anno, lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0, pad_type='edge'):
    p1 = np.array((anno['fm1x'], anno['fm1y'])).astype('f')
    p2 = np.array((anno['fm0x'], anno['fm0y'])).astype('f')
    face_width = anno['y2'] - anno['y1']
    angle = angle_between_2_points(p1, p2)
    x1, y1 = p1
    x2, y2 = p2
    xc = (x1 + x2) // 2
    yc = (y1 + y2) // 2
    dis_width = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 2.0
    pad_size = max(im.size[0], im.size[1]) / 2
    np_im = np.array(im)
    if pad_type=='constant':
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type, constant_values=0), \
                                          np.pad(np_im[:,:,1], pad_size, pad_type, constant_values=0), \
                                          np.pad(np_im[:,:,2], pad_size, pad_type, constant_values=0)]).T, 3))
    else:
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type ), \
                                  np.pad(np_im[:,:,1], pad_size, pad_type), \
                                  np.pad(np_im[:,:,2], pad_size, pad_type)]).T, 3))
    tmp_im = ImageOps.mirror(tmp_im)
    xc = xc + pad_size
    yc = yc + pad_size
    tmp_im = tmp_im.rotate(angle, center=(xc, yc), resample=Image.BICUBIC)
    w = face_width
    h = w / lambda1 * lambda2
    x1 = anno['y1'] - w/2 + pad_size
    y1 = yc - w / lambda1 * lambda3
    x2 = x1 + 2*w
    y2 = y1 + h
    return tmp_im.crop((x1,y1,x2,y2)).resize((178,218), resample=Image.BICUBIC)

def align_eye_pad_ailab_show(im, anno, lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0, pad_type='edge', zoom=0.1):
    p1 = np.array((anno['fm1x'], anno['fm1y'])).astype('f')
    p2 = np.array((anno['fm0x'], anno['fm0y'])).astype('f')
    face_width = anno['y2'] - anno['y1']
    angle = angle_between_2_points(p1, p2)
    x1, y1 = p1
    x2, y2 = p2
    xc = (x1 + x2) // 2
    yc = (y1 + y2) // 2
    dis_width = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 2.0
    pad_size = max(im.size[0], im.size[1]) / 2
    np_im = np.array(im)
    if pad_type=='constant':
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type, constant_values=0), \
                                          np.pad(np_im[:,:,1], pad_size, pad_type, constant_values=0), \
                                          np.pad(np_im[:,:,2], pad_size, pad_type, constant_values=0)]).T, 3))
    else:
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type ), \
                                  np.pad(np_im[:,:,1], pad_size, pad_type), \
                                  np.pad(np_im[:,:,2], pad_size, pad_type)]).T, 3))
    tmp_im = ImageOps.mirror(tmp_im)
    xc = xc + pad_size
    yc = yc + pad_size
    tmp_im = tmp_im.rotate(angle, center=(xc, yc), resample=Image.BICUBIC)
    w = face_width
    h = w / lambda1 * lambda2
    x1 = anno['y1'] - w/2 + pad_size
    y1 = yc - w / lambda1 * lambda3
    xd = 2*w 
    yd = h
    adj = yd * zoom
    return tmp_im.crop(((x1)-((yd-xd)/2)+adj,y1+adj,(x1)-((yd-xd)/2)+yd-adj,y1+yd-adj)).resize((128,128), resample=Image.BICUBIC)

def face_detect(img_path, gpu_id=0):
    old_ld_library = os.environ['LD_LIBRARY_PATH']
    os.environ['LD_LIBRARY_PATH'] = './util/face/lib:./util/face/public_libs:/usr/local/caffe/lib:'+os.environ['LD_LIBRARY_PATH']
    tmp = subprocess.check_output(['./util/face/AILab_FaceDemo', img_path, './util/face/models/', str(gpu_id)])
    os.environ['LD_LIBRARY_PATH'] = old_ld_library
    return eval('{'+''.join(tmp.split('\n')[1:16])+'}')

def test_lambda_2( lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0):
    img_path = '/data2/minjunli/prj/anime/face_align/test/tmp.jpg'
    print img_path
    attr = face_detect(img_path)
    print attr
    im = Image.open(img_path).convert('RGB')
    B_ =align_eye_pad_ailab_show(im,attr, lambda1, lambda2, lambda3)
    fake_A_1 = test_img(B_, net_1['B'], img_pre_128, 128, eval_mode=True)
    fake_A_2 = test_img(B_, net_2['B'], img_pre_128, 128, eval_mode=True)
    fake_A_5 = test_img(B_, net_5['B'], img_pre_128, 128, eval_mode=True)
    fake_A_6 = test_img(B_, net_6['B'], img_pre_128, 128, eval_mode=True)
    show(torchvision.utils.make_grid([img_pre_128(im), img_pre_128(B_), fake_A_1.data[0].cpu(), fake_A_2.data[0].cpu(), fake_A_5.data[0].cpu(), fake_A_6.data[0].cpu()]))

interact(test_lambda_2,  lambda1=(70.0,100.0,1),  lambda2=(210.0,230.0,1),  lambda3=(100.0,120.0,1))


In [ ]:
img_path ='/data2/minjunli/prj/anime/face_lib_points/test/nan2.jpg'
print img_path
attr = face_detect(img_path)
print attr
im = Image.open(img_path).convert('RGB')
lambda1 = 77.0
lambda2 = 228.0
lambda3 = 111.0
align_eye_pad_ailab_show(im,attr, lambda1, lambda2, lambda3)

In [ ]:
import os, sys
import glob
list_ = glob.glob('../dst_men/*')
lists = []
for idx, i in enumerate(list_):
    lists.append((i, idx%7))

In [ ]:
i = lists[0]
img_path = i[0]
img_name = img_path.split('/')[-1]
attr = face_detect(img_path, i[1])
im = Image.open(img_path).convert('RGB')
lambda1 = 77.0
lambda2 = 228.0
lambda3 = 111.0
align_eye_pad_ailab(im,attr, lambda1, lambda2, lambda3).save('../dst_men_aligned/'+img_name)

In [ ]:
img_path

In [ ]:
from multiprocessing import Pool

def test(i):
    img_path = i[0]
    img_name = img_path.split('/')[-1]
    attr = face_detect(img_path, i[1])
    im = Image.open(img_path).convert('RGB')
    lambda1 = 77.0
    lambda2 = 228.0
    lambda3 = 111.0
    align_eye_pad_ailab(im,attr, lambda1, lambda2, lambda3).save('../dst_men_aligned/'+img_name)

pool=Pool(processes=28) 
pool.map(test,lists)
pool.close()
pool.join()

In [ ]:
B_path = glob.glob('./datasets/getchu_aisacelebaselfie_aligned_cleaned_masked_blur/trainB/*')

In [ ]:
def predict_mask(img):
    image_width =  None
    image_height = None

    image_base64 = _to_img(img, image_width, image_height)

    r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})
    print r.status_code
    #print r.content
    js = r.json()
    #print js
    mask = _get_img(js['prob'], image_width, image_height)
    #image_matting = _get_img(js['img_data'], image_width, image_height)

    img_npy = np.asarray(img)

    mask = mask.point(lambda p: p > 50 and 255) 
    #mask = mask.point(lambda p: p < 128 or 0)

    mask_npy = np.asarray(mask) / 255

    mask_inv_npy = 1 - mask_npy
    white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255


    img_masked_npy = np.multiply(img_npy, mask_npy) + np.multiply(white_img_npy, mask_inv_npy)
    img_masked = Image.fromarray(img_masked_npy)
    return img_masked

def align(img, attr, lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0):
    img_aligned =align_eye_pad_ailab(img,attr, lambda1, lambda2, lambda3)
    return img_aligned
    

def align_show(img, attr, lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0):
    img_aligned =align_eye_pad_ailab_show(img,attr, lambda1, lambda2, lambda3)
    return img_aligned
    
img_path = random.choice(B_path)
# attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = img
#img_aligned = align(img, attr)

fake_A_1 = test_img(img_aligned, net_1['B'], img_pre_128, 128, bn_eval=True, drop_eval=False)
fake_A_2 = test_img(img_aligned, net_12['B'], img_pre_128, 128, bn_eval=True, drop_eval=False)
tmp = show(torchvision.utils.make_grid([img_pre_128(img), img_pre_128(img_aligned), fake_A_1.data[0].cpu(), fake_A_2.data[0].cpu()]),show=True)

# img_path = './datasets/getchu_aisacelebaselfie_aligned_cleaned_masked_blur_v6/trainB/'+img_path.split('/')[-1]
# attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = img
#img_aligned = align(img, attr)

fake_A_5 = test_img(img_aligned, net_1['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
fake_A_6 = test_img(img_aligned, net_12['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
tmp = show(torchvision.utils.make_grid([img_pre_128(img), img_pre_128(img_aligned), fake_A_5.data[0].cpu(), fake_A_6.data[0].cpu()]),show=True)


fake_A_1 = test_img(img_aligned, net_1['B'], img_pre_128, 128, bn_eval=True, drop_eval=True)
fake_A_2 = test_img(img_aligned, net_12['B'], img_pre_128, 128, bn_eval=True, drop_eval=True)
tmp = show(torchvision.utils.make_grid([img_pre_128(img), img_pre_128(img_aligned), fake_A_1.data[0].cpu(), fake_A_2.data[0].cpu()]),show=True)

# img_path = './datasets/getchu_aisacelebaselfie_aligned_cleaned_masked_blur_v6/trainB/'+img_path.split('/')[-1]
# attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = img
#img_aligned = align(img, attr)

fake_A_5 = test_img(img_aligned, net_1['B'], img_pre_128, 128, bn_eval=False, drop_eval=False)
fake_A_6 = test_img(img_aligned, net_12['B'], img_pre_128, 128, bn_eval=False, drop_eval=False)
tmp = show(torchvision.utils.make_grid([img_pre_128(img), img_pre_128(img_aligned), fake_A_5.data[0].cpu(), fake_A_6.data[0].cpu()]),show=True)



In [ ]:
test_set = []

In [ ]:
import json
data_json = json.load(open('../fake_detect/data.json'))
rev_attr = {}
for i in data_json:
    rev_attr[i['ceph_path'].split('/')[-1]] = i

In [ ]:
idx = 19

In [ ]:
Image.open('../fake_detect/haozhi_val/'+data_json[idx]['ceph_path'].split('/')[-1]).resize((128,128))

In [ ]:
test_set = ['../fake_detect/haozhi_val/3412c89849f816f68a2caecfdec85cd2.jpg',
 '../fake_detect/haozhi_val/9bbff8f026e11b964dd3f363c8167a1a.jpg',
 '../fake_detect/haozhi_val/c876b69809abb6e3a6b040acd9d85d06.jpg',
 '../fake_detect/haozhi_val/3818efb9fb4a31e1dfb9de3cc07a65b1.jpg',
 '../fake_detect/haozhi_val/ccdb709e62d2cdb347a04c368c13cbf6.jpg',
 '../fake_detect/haozhi_val/dd30dcb520c2a960b5b5818f4f4c688d.jpg',
 '../fake_detect/haozhi_val/d599ea74388468955e4781a38e6cf340.jpg',
 '../fake_detect/haozhi_val/59a917cc6b4180bbf7fc6ce52a2ffa61.jpg',
 '../fake_detect/haozhi_val/8dbd4b21667951908e42c989226d68ec.jpg',
 '../fake_detect/haozhi_val/a6a4f6b4e8d8fb9f59cb3abf15aef259.jpg',
 '../fake_detect/haozhi_val/b26f66e83d68d25c63df2665ff0ba833.jpg',
 '../fake_detect/haozhi_val/06b9a10f14c38183c6d716086d7dde37.jpg',
 '../fake_detect/haozhi_val/863637bae9ff973129205eea6bdc3a91.jpg',
 '../fake_detect/haozhi_val/12905b2072e761696be4cbccf0b1c647.jpg',
 '../fake_detect/haozhi_val/23c163554dee92751d7ea557c677650d.jpg',
 '../fake_detect/haozhi_val/bf3b1877667037662644d15e7ad168c5.jpg',
 '../fake_detect/haozhi_val/677a444a66ff02568a451318c03c9280.jpg',
 '../fake_detect/haozhi_val/8ea7d2925deac43e8394da1d83c299d7.jpg',
 '../fake_detect/haozhi_val/1f7bcec7ab4c2edaa11fca9de813a00e.jpg',
 '../fake_detect/haozhi_val/fd50522b693b5796385e76c7f7bfd355.jpg',
 '../fake_detect/haozhi_val/ef45020bd3c87e08dc958e19b7824a49.jpg',
 '../fake_detect/haozhi_val/9a844f36507160473f50b5ae0462bba0.jpg',
 '../fake_detect/haozhi_val/5b577804f97ce742ff74dfb26b164d83.jpg',
 '../mytestimg/myimg/P81503-3069625407.1456271473.jpg',
 '../face_lib_points/testimg/nan1_aligned.jpg']


In [ ]:
for idx, img_path in enumerate(tqdm.tqdm_notebook(test_set)):
    attr = face_detect(img_path)
    img = Image.open(img_path).convert('RGB')
    img_aligned = align(img, attr)
    img_show = align_show(img, attr)
    img_masked = predict_mask_alt(img_aligned)
    test_net = range(37)

    img_show.save('./user_study/orig_%d.png'%idx)
    model_count = 0
    dropout = set([7,4,8,9,13,15,12,16,24,25,26,35,32,28,29,18,31,36,23])
    for i in test_net:
        exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

    for i in test_net:
        if i not in dropout:
            show(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i),show=False).save('./user_study/trans_%d_%d.png'%(idx,model_count))
            model_count += 1

    
    dropout = set([8,9,13,26,16,28,29,5,35,12,19,18,7,25,10])

    for i in test_net:
        exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

    for i in test_net:
        if i not in dropout:
            show(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i),show=False).save('./user_study/trans_%d_%d.png'%(idx,model_count))
            model_count += 1


In [ ]:
import tqdm
test_set_ex = test_set
for idx in tqdm.tqdm_notebook(range(25, 200)):
    img_path = random.choice(B_path)
    attr = face_detect(img_path)
    while rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1 or ('fm1x' not in attr.keys()) or img_path in test_set_ex:
        img_path = random.choice(B_path)
        attr = face_detect(img_path)
    test_set_ex.append(img_path)
    img = Image.open(img_path).convert('RGB')
    img_aligned = align(img, attr)
    img_show = align_show(img, attr)
    img_masked = predict_mask_alt(img_aligned)
    test_net = range(37)

    img_show.save('./user_study/orig_%d.png'%idx)
    model_count = 0
    dropout = set([7,4,8,9,13,15,12,16,24,25,26,35,32,28,29,18,31,36,23])
    for i in test_net:
        exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

    for i in test_net:
        if i not in dropout:
            show(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i),show=False).save('./user_study/trans_%d_%d.png'%(idx,model_count))
            model_count += 1

    
    dropout = set([8,9,13,26,16,28,29,5,35,12,19,18,7,25,10])

    for i in test_net:
        exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

    for i in test_net:
        if i not in dropout:
            show(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i),show=False).save('./user_study/trans_%d_%d.png'%(idx,model_count))
            model_count += 1

In [ ]:
import tqdm
count_idx = 25
for img_path in tqdm.tqdm_notebook(filtered_list[:25]):
    attr = face_detect(img_path)
    while rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1 or ('fm1x' not in attr.keys()) \
        or os.path.join('../fake_detect/haozhi_val',img_path.split('/')[-1]) in test_set:
        continue
    img = Image.open(img_path).convert('RGB')
    img_aligned = align(img, attr)
    img_show = align_show(img, attr)
    img_masked = predict_mask_alt(img_aligned)
    test_net = range(37)

    img_show.save('./user_study/orig_%d.png'%count_idx)
    model_count = 0
    dropout = set([7,4,8,9,13,15,12,16,24,25,26,35,32,28,29,18,31,36,23])
    for i in test_net:
        exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

    for i in test_net:
        if i not in dropout:
            show(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i),show=False).save('./user_study/trans_%d_%d.png'%(count_idx,model_count))
            model_count += 1

    
    dropout = set([8,9,13,26,16,28,29,5,35,12,19,18,7,25,10])

    for i in test_net:
        exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

    for i in test_net:
        if i not in dropout:
            show(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i),show=False).save('./user_study/trans_%d_%d.png'%(count_idx,model_count))
            model_count += 1
    count_idx += 1

In [ ]:
count_idx = 24
for img_path in filtered_list:
    print img_path
    attr = face_detect(img_path)
    if not(rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1 or ('fm1x' not in attr.keys()) \
        or os.path.join('../fake_detect/haozhi_val',img_path.split('/')[-1]) in test_set):
        count_idx += 1
        img = Image.open(img_path).convert('RGB')
        img_aligned = align(img, attr)
        img_show = align_show(img, attr)
        img_masked = predict_mask_alt(img_aligned)
        test_net = range(37)

        img_show.save('./user_study/orig_%d.png'%count_idx)
        model_count = 0
        dropout = set([7,4,8,9,13,15,12,16,24,25,26,35,32,28,29,18,31,36,23])
        for i in test_net:
            exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

        for i in test_net:
            if i not in dropout:
                show(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i),show=False).save('./user_study/trans_%d_%d.png'%(count_idx,model_count))
                model_count += 1

        to_show = [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show)]
        for i in range(7):
            to_show.append(img_pre_128(blank_img))
        for i in test_net:
            if i not in dropout:
                to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i))
            else:
                to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),) / 3'%i))

        _ = show(torchvision.utils.make_grid(to_show,nrow=10),show=True)


        dropout = set([8,9,13,26,16,28,29,5,35,12,19,18,7,25,10])

        for i in test_net:
            exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

        for i in test_net:
            if i not in dropout:
                show(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i),show=False).save('./user_study/trans_%d_%d.png'%(count_idx,model_count))
                model_count += 1


In [ ]:
remain_model = []

test_net = range(37)

model_count = 0
dropout = set([7,4,8,9,13,15,12,16,24,25,26,35,32,28,29,18,31,36,23])

for i in test_net:
    if i not in dropout:
        remain_model.append((0, i))
        model_count += 1

dropout = set([8,9,13,26,16,28,29,5,35,12,19,18,7,25,10])

for i in test_net:
    if i not in dropout:
        remain_model.append((1, i))
        model_count += 1


In [ ]:
[remain_model[i] for i in sorted_idx]

In [ ]:
fake_A_0.data[0].cpu()[img_pre_128(img_aligned)==1.0]=1.0

In [ ]:
import json
attrs = json.load(open('./tmp/data.json'))

In [ ]:
Image.open(keylist[0])


In [ ]:
class timeit():
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

In [ ]:
%%time
len(remain_model)
sorted_idx =[ 5,  4, 15,  8, 22, 23, 13, 29,  3, 14, 35, 10, 11, 28, 12, 19, 31,
              6, 16,  2,  7,  0, 24, 18, 37, 25, 34, 27, 39, 20, 30, 33,  9,  1,
             36, 21, 26, 17, 32, 38]
#keylist = sorted(attrs.keys())
attr = attrs[keylist[0]]
img = Image.open(img_path).convert('RGB')
img_aligned = align_eye_pad_ailab(img, attr, pad_type='edge')
crop_pos, orig_pos, img_show = align_eye_pad_ailab_show_alt(img, attr, zoom=0.1, show=False)
img_masked = predict_mask_alt(align_eye_pad_ailab(img, attr))

for idx, i in enumerate(sorted_idx[-4:]):
    if remain_model[i][0] == 0:
        exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))
    else:
        exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))

for i in range(4):
    show(eval("post_process(fake_A_%d.data[0].cpu(), orig_pos=orig_pos)"%i),show=False)
    show(eval("post_process(fake_A_%d.data[0].cpu())"%i),show=False)

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset, DataLoader

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1) 

class NLayerDiscriminatorMod(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=3, max_mult=8, norm_layer=nn.BatchNorm2d, feat_len=256):
        super(NLayerDiscriminatorMod, self).__init__()
        use_bias = False
        kw = 3

        padw = 1
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=7, stride=2, padding=3),
            norm_layer(ndf),
            nn.LeakyReLU(0.2, True)
        ]

        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, max_mult)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                          kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult)]
            sequence += [nn.LeakyReLU(0.2, True)]

        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, max_mult)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                      kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]
        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=3, stride=1, padding=padw)]
        # sequence += [nn.AvgPool2d(7, stride=1)]
        # sequence += [nn.Linear(feat_len, 1)]
        sequence += [Flatten(), nn.Linear(feat_len, 1)]

        sequence += [nn.Sigmoid()]

        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        return self.model(input)


def load_dis(path):
    dis_model = NLayerDiscriminatorMod(3, ndf=16, n_layers = 6, max_mult=8, feat_len=4)
    dis_model = dis_model.cuda()
    checkpoint = torch.load(path)
    dis_model.load_state_dict(checkpoint['state_dict'])
    dis_model.eval()
    return dis_model



# dis_model_0_b = load_dis('../fake_detect/train/dis_mod_v2_0_balanced/chkpt_7700.pth.tar')
# dis_model_1_b = load_dis('../fake_detect/train/dis_mod_v2_1_balanced/chkpt_10700.pth.tar')
# dis_model_2_b = load_dis('../fake_detect/train/dis_mod_v2_2_balanced/chkpt_7000.pth.tar')
# dis_model_3_b = load_dis('../fake_detect/train/dis_mod_v2_3_balanced/chkpt_6700.pth.tar')
# dis_model_0_n = load_dis('../fake_detect/train/dis_mod_v2_0_normal/chkpt_4800.pth.tar')
# dis_model_1_n = load_dis('../fake_detect/train/dis_mod_v2_1_normal/chkpt_6500.pth.tar')
# dis_model_2_n = load_dis('../fake_detect/train/dis_mod_v2_2_normal/chkpt_5600.pth.tar')
# dis_model_3_n = load_dis('../fake_detect/train/dis_mod_v2_3_normal/chkpt_4800.pth.tar')

dis_model_all_v1 = load_dis('../fake_detect/train/dis_mod_v2_all_balanced/chkpt_2300.pth.tar')
dis_model_all_v2 = load_dis('../fake_detect/train/dis_mod_v3_all_balanced_cont/chkpt_2100.pth.tar')

In [ ]:
len(remain_model)
sorted_idx =[ 5,  4, 15,  8, 22, 23, 13, 29,  3, 14, 35, 10, 11, 28, 12, 19, 31,
              6, 16,  2,  7,  0, 24, 18, 37, 25, 34, 27, 39, 20, 30, 33,  9,  1,
             36, 21, 26, 17, 32, 38]

# B_path = glob.glob('../fake_detect/haozhi_val/*')
# img_path = './FullSizeRender_(1).jpg'
img_path =  random.choice(B_path)#'../face_lib_points/testimg/xsichen.jpg'#random.choice(B_path)
while rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1:
    img_path = random.choice(B_path)
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = align_eye_pad_ailab(img, attr, pad_type='edge')
crop_pos, orig_pos, img_show = align_eye_pad_ailab_show_alt(img, attr, zoom=0.1, show=True)
#img_aligned = img_show
print crop_pos
img_masked = predict_mask_alt(img_aligned)
predict_mask_raw(img_aligned).resize((128,128))

In [ ]:
img_masked.resize((128,128))

In [ ]:
img_aligned.resize((128,128))

In [ ]:
len(remain_model)
sorted_idx =[ 5,  4, 15,  8, 22, 23, 13, 29,  3, 14, 35, 10, 11, 28, 12, 19, 31,
              6, 16,  2,  7,  0, 24, 18, 37, 25, 34, 27, 39, 20, 30, 33,  9,  1,
             36, 21, 26, 17, 32, 38]
B_path = glob.glob('../fake_detect/haozhi_val/*')
# img_path = './FullSizeRender_(1).jpg'
img_path = '/data2/minjunli/1.png' # random.choice(B_path)
# while rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1:
#     img_path = random.choice(B_path)
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = align_eye_pad_ailab(img, attr, pad_type='edge')
crop_pos, orig_pos, img_show = align_eye_pad_ailab_show_alt(img, attr, zoom=0.1, show=True)
#img_aligned = img_show
print crop_pos
img_masked = predict_mask_alt(img_aligned)

for idx, i in enumerate(sorted_idx[-4:]):
    if remain_model[i][0] == 0:
        exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))
    else:
        exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))

blank_img = Image.new('RGB',(128,128),(255,255,255))

to_show = [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show), img_pre_128(img_masked)]
# for i in range(6):
#     to_show.append(img_pre_128(blank_img))

bad_th = [0.5, 0.5, 0.5, 0.5]

scores = []
for i in range(len(sorted_idx[:4])):
    dis_model_all_v1.eval()
    score = eval('dis_model_all_v1(fake_A_%d)'%(i))
    scores.append(score.data.cpu().numpy())
    if score.data.cpu().numpy() > bad_th[i]:
        to_show.append(eval('post_process(fake_A_%d.data[0].cpu(),crop_pos=crop_pos)'%i))
    else:
        to_show.append(eval('post_process(fake_A_%d.data[0].cpu(),crop_pos=crop_pos) / 3'%i))

# for i in range(len(sorted_idx[:4])):
#     score = eval('dis_model_%d_b(fake_A_%d)'%(i,i))
#     scores.append(score.data.cpu().numpy())
#     if score.data.cpu().numpy() > bad_th[i]:
#         to_show.append(eval('post_process(fake_A_%d.data[0].cpu(),crop_pos=crop_pos)'%i))
#     else:
#         to_show.append(eval('post_process(fake_A_%d.data[0].cpu(),crop_pos=crop_pos) / 3'%i))
#     to_show.append(eval('fake_A_%d.data[0].cpu()'%i))

############## ft

for i in reversed(range(44,52)):
    exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

for i in reversed(range(44,52)):
    to_show.append(eval('post_process(fake_A_%d.data[0].cpu(), crop_pos=crop_pos)'%i))
    
for i in (range(37,43)):
    exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))
    to_show.append(eval('post_process(fake_A_%d.data[0].cpu(), crop_pos=crop_pos)'%i))

# to_show.append(img_pre_128(blank_img))
# to_show.append(img_pre_128(blank_img))

for i in range(3,11):
    exec('fake_A_47_%d = test_img(img_masked, net_47_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))
    to_show.append(eval('post_process(fake_A_47_%d.data[0].cpu(), crop_pos=crop_pos)'%i))

for i in range(3,11):
    exec('fake_A_49_%d = test_img(img_masked, net_49_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))
    to_show.append(eval('post_process(fake_A_49_%d.data[0].cpu(), crop_pos=crop_pos)'%i))
    
for i in range(3,11):
    exec('fake_A_51_%d = test_img(img_masked, net_51_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))
    to_show.append(eval('post_process(fake_A_51_%d.data[0].cpu(), crop_pos=crop_pos)'%i))
    
############ ft end    

# img_aligned = align_eye_pad_ailab(img, attr, pad_type='constant')
# img_show = align_eye_pad_ailab_show(img, attr, pad_type='constant')
# #img_aligned = img_show
# img_masked = predict_mask_alt(align_eye_pad_ailab(img, attr))

# to_show += [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show), img_pre_128(img_masked)]

# for idx, i in enumerate(sorted_idx):
#     if remain_model[i][0] == 0:
#         exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))
#     else:
#         exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))

# for i in range(36,40):
#     to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i))
# #     to_show.append(eval('fake_A_%d.data[0].cpu()'%i))

# for j in range(0,20,5):
#     to_show += [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show), img_pre_128(img_masked)]
#     for i in range(36,40):
#         to_show.append(eval('post_process(fake_A_%d.data[0].cpu(),img_aligned,row_del=%d)'%(i,j)))

show(torchvision.utils.make_grid(to_show,nrow=8),show=True)

for i in range(1, 9):
    print '%.3f\t'%scores[i-1][0][0],
    if i % 4 == 0:
        print ''


In [ ]:
img_aligned

In [ ]:
Image.open(img_path)

In [ ]:
len(remain_model)
sorted_idx =[ 5,  4, 15,  8, 22, 23, 13, 29,  3, 14, 35, 10, 11, 28, 12, 19, 31,
              6, 16,  2,  7,  0, 24, 18, 37, 25, 34, 27, 39, 20, 30, 33,  9,  1,
             36, 21, 26, 17, 32, 38]

#avg_scores = []
# for i in range(40):
#     avg_scores.append([])

for ii in tqdm.tqdm_notebook(range(100)):
    img_path =  random.choice(B_path)
    while rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1:
        img_path = random.choice(B_path)
    attr = face_detect(img_path)
    if 'fm1x' not in attr.keys():
        continue
    img = Image.open(img_path).convert('RGB')
    img_aligned = align_eye_pad_ailab(img, attr, pad_type='edge')
    crop_pos, orig_pos, img_show = align_eye_pad_ailab_show_alt(img, attr, zoom=0.1, show=True)
    #img_aligned = img_show
    print crop_pos
    img_masked = predict_mask_alt(img_aligned)

    for idx, i in enumerate(sorted_idx):
        if remain_model[i][0] == 0:
            exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))
        else:
            exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))


    for i in range(len(sorted_idx)):
        score = dis_model(eval('fake_A_%d'%i))
        avg_scores[i].append(score.data.cpu().numpy()[0][0])

for i in range(1, 41):
    print '%.3f\t'%(np.array(avg_scores[i-1]).sum() / np.array(avg_scores[i-1]).size),
    if i % 10 == 0:
        print ''

In [ ]:
print np.array(avg_scores[0]).size
for i in range(1, 41):
    print '%.3f\t'%(np.array(avg_scores[i-1]).sum() / np.array(avg_scores[i-1]).size),
    if i % 10 == 0:
        print ''

In [ ]:
im = Image.open(img_path)
im.save('../face_align/test/masktest.png')
predict_mask_raw(im).save('../face_align/test/masktest_mask.png')

In [ ]:
def predict_mask_raw(img):
    image_width =  None
    image_height = None

    image_base64 = _to_img(img, image_width, image_height)

    r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})

    #print r.content
    js = r.json()
    #print js
    mask = _get_img(js['prob'], image_width, image_height)
    #image_matting = _get_img(js['img_data'], image_width, image_height)
#     img_npy = np.asarray(img)
#     white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255
#     softmask = (1-np.array(mask.filter(ImageFilter.GaussianBlur))/255.0)
#     masked_img = img * (1-softmask) + white_img_npy * (softmask)

#     return Image.fromarray(masked_img.astype(np.uint8))
    return mask

def mask_img(mask, img):
    img_npy = np.asarray(img)
    white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255
    softmask = (1-np.array(mask.filter(ImageFilter.GaussianBlur))/255.0)
    masked_img = img * (1-softmask) + white_img_npy * (softmask)

    return Image.fromarray(masked_img.astype(np.uint8))

sorted_idx =[ 5,  4, 15,  8, 22, 23, 13, 29,  3, 14, 35, 10, 11, 28, 12, 19, 31,
              6, 16,  2,  7,  0, 24, 18, 37, 25, 34, 27, 39, 20, 30, 33,  9,  1,
             36, 21, 26, 17, 32, 38]

# B_path = glob.glob('../fake_detect/haozhi_val/*')
# img_path = './FullSizeRender_(1).jpg'
img_path = random.choice(B_path)
# while rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1:
#     img_path = random.choice(B_path)
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
mask_raw = predict_mask_raw(img)
img_aligned = align_eye_pad_ailab(img, attr, pad_type='edge')
mask_aligned = align_eye_pad_ailab(mask_raw, attr, pad_type='constant')
crop_pos, orig_pos, img_show = align_eye_pad_ailab_show_alt(img, attr, zoom=0.1, show=True)
#img_aligned = img_show
print crop_pos
img_masked = mask_img(mask_aligned, img_aligned)

for idx, i in enumerate(sorted_idx[-8:]):
    if remain_model[i][0] == 0:
        exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))
    else:
        exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))

blank_img = Image.new('RGB',(128,128),(255,255,255))

to_show = [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show), img_pre_128(img_masked)]
for i in range(4):
    to_show.append(img_pre_128(blank_img))

for i in range(8):
    to_show.append(eval('post_process(fake_A_%d.data[0].cpu(),crop_pos=crop_pos)'%i))
    to_show.append(img_pre_128(blank_img))
#     to_show.append(eval('fake_A_%d.data[0].cpu()'%i))

for i in reversed(range(44,52)):
    exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

for i in reversed(range(44,52)):
    to_show.append(eval('post_process(fake_A_%d.data[0].cpu(), crop_pos=crop_pos)'%i))
    
for i in (range(37,43)):
    exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))
    to_show.append(eval('post_process(fake_A_%d.data[0].cpu(), crop_pos=crop_pos)'%i))

to_show.append(img_pre_128(blank_img))
to_show.append(img_pre_128(blank_img))

for i in range(3,11):
    exec('fake_A_47_%d = test_img(img_masked, net_47_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))
    to_show.append(eval('post_process(fake_A_47_%d.data[0].cpu(), crop_pos=crop_pos)'%i))

for i in range(3,11):
    exec('fake_A_49_%d = test_img(img_masked, net_49_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))
    to_show.append(eval('post_process(fake_A_49_%d.data[0].cpu(), crop_pos=crop_pos)'%i))
    
for i in range(3,11):
    exec('fake_A_51_%d = test_img(img_masked, net_51_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))
    to_show.append(eval('post_process(fake_A_51_%d.data[0].cpu(), crop_pos=crop_pos)'%i))
    
# img_aligned = align_eye_pad_ailab(img, attr, pad_type='constant')
# img_show = align_eye_pad_ailab_show(img, attr, pad_type='constant')
# #img_aligned = img_show
# img_masked = predict_mask_alt(align_eye_pad_ailab(img, attr))

# to_show += [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show), img_pre_128(img_masked)]

# for idx, i in enumerate(sorted_idx):
#     if remain_model[i][0] == 0:
#         exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))
#     else:
#         exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(idx, remain_model[i][1]))

# for i in range(36,40):
#     to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i))
# #     to_show.append(eval('fake_A_%d.data[0].cpu()'%i))

# for j in range(0,20,5):
#     to_show += [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show), img_pre_128(img_masked)]
#     for i in range(36,40):
#         to_show.append(eval('post_process(fake_A_%d.data[0].cpu(),img_aligned,row_del=%d)'%(i,j)))

show(torchvision.utils.make_grid(to_show,nrow=8),show=False)


In [ ]:

sorted_idx =[ 5,  4, 15,  8, 22, 23, 13, 29,  3, 14, 35, 10, 11, 28, 12, 19, 31,
              6, 16,  2,  7,  0, 24, 18, 37, 25, 34, 27, 39, 20, 30, 33,  9,  1,
             36, 21, 26, 17, 32, 38]

# B_path = glob.glob('../fake_detect/haozhi_val/*')
# img_path = './FullSizeRender_(1).jpg'
show_o = []
show_a = {}
for i in range(3):
    show_a[i] = []

for img_path in sorted(glob.glob('./report_show/gif1/*'))[:16]:
    attr = face_detect(img_path)
    img = Image.open(img_path).convert('RGB')
    img_aligned = align_eye_pad_ailab(img, attr, pad_type='edge')
    crop_pos, orig_pos, img_show = align_eye_pad_ailab_show_alt(img, attr, zoom=0.1, show=True)
    print crop_pos
    img_masked = predict_mask_alt(img_aligned)

    for i in range(3):
        exec('fake_cont = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i))
        show_a[i].append(eval('post_process(fake_cont.data[0].cpu(),crop_pos=crop_pos)'))
#     for idx, i in enumerate(sorted_idx[-1:]):
#         if remain_model[i][0] == 0:
#             exec('fake_cont = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(remain_model[i][1]))
#         else:
#             exec('fake_cont = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(remain_model[i][1]))

    blank_img = Image.new('RGB',(128,128),(255,255,255))

    show_o.append(img_pre_128(img_show))
    

_ = show(torchvision.utils.make_grid(show_o,nrow=8),show=True)
for i in range(3):
    _ = show(torchvision.utils.make_grid(show_a[i],nrow=8),show=True)


In [ ]:
show(post_process(fake_A_6.data[0].cpu(),crop_pos=crop_pos),show=False).save('./report_show_new/3_0.png')

In [ ]:
show(img_pre_256(img_show),show=False).save('./report_show_new/3_orig.png')

In [ ]:
def post_process(img, orig_pos=None, crop_pos=None, col_add=0, row_del=0):
    t = img
    to_show_pre = transforms.Compose([transforms.Scale((128,128), Image.BICUBIC)
                                      ,transforms.ToTensor()
                                      ,transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
    if orig_pos is not None:
        mask = Image.new('RGB', (128,128), (255,255,255))
        draw = ImageDraw.Draw(mask)
        draw.polygon(orig_pos, fill=(0,0,0))
        del draw
        t[img_pre_128(mask)==1.0]=1.0
    if crop_pos is not None:
        padding = int((crop_pos[1][0] - crop_pos[0][0]) * 0.04)
        x1 = max(int(crop_pos[0][0]) - padding, 0)
        x2 = min(int(crop_pos[1][0]) + padding, 128)
        y1 = max(int(crop_pos[0][1]) + padding, 0)
        y2 = min(int(crop_pos[1][1]) - padding, 128)
        t = t[::,y1:y2,x1:x2]
    else:
        pass
#         padding = int(128 * 0.07)
#         x1 = padding
#         x2 = 128 - padding
#         y1 = 0
#         y2 = 128
#         print x1, x2, y1, y2
#         t = t[::,y1:y2,x1:x2]
    if col_add != 0:
        tmp = torch.ones(3, 128, col_add)
        t = torch.cat((tmp, t, tmp), 2)
    if row_del != 0:
        t = t[::,row_del:(128-row_del)]
    return to_show_pre(Image.fromarray(util.tensor2im(t.unsqueeze(0))).resize((256,256),Image.LANCZOS).resize((128,128),Image.LANCZOS))

show(post_process(fake_A_2.data[0].cpu(), crop_pos = crop_pos),show=False)

In [ ]:
crop_pos

In [ ]:
fake_A_0.data[0].cpu()[::,10:110]

In [ ]:
from PIL import Image, ImageDraw

import math

def distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 )

def rotate_point(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point
    radius = np.radians(angle)
    qx = ox + math.cos(radius) * (px - ox) - math.sin(radius) * (py - oy)
    qy = oy + math.sin(radius) * (px - ox) + math.cos(radius) * (py - oy)
    return qx, qy

def draw_point(draw, p, r=20, fill=(255,0,0)):
    if p is None:
        return
    x, y = p
    draw.ellipse(((x-r), (y-r), (x+r), (y+r)), fill=fill)

def angle_between_2_points(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    tan = (y2 - y1) / (x2 - x1)
    return np.degrees(np.arctan(tan))
    
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1]) #Typo was here

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y    

def point_in_rect(p, p1, p2):
    return p[0] <= p2[0] and p[0] >= p1[0] and  p[1] <= p2[1] and p[1] >= p1[1]

def align_eye_pad_ailab_show_alt(im, anno, lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0, zoom=0.1, show=True):
    p1 = np.array((anno['fm1x'], anno['fm1y'])).astype('f')
    p2 = np.array((anno['fm0x'], anno['fm0y'])).astype('f')
    face_width = anno['y2'] - anno['y1']
    angle = angle_between_2_points(p1, p2)
    x1, y1 = p1
    x2, y2 = p2
    xc = (x1 + x2) // 2
    yc = (y1 + y2) // 2
    dis_width = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 2.0
    pad_type = 'edge'
    pad_size = max(im.size[0], im.size[1]) / 2
    np_im = np.array(im)
    if pad_type == 'constant':
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type, constant_values=255), \
                                          np.pad(np_im[:,:,1], pad_size, pad_type, constant_values=255), \
                                          np.pad(np_im[:,:,2], pad_size, pad_type, constant_values=255)]).T, 3))
    else:
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type), \
                                  np.pad(np_im[:,:,1], pad_size, pad_type), \
                                  np.pad(np_im[:,:,2], pad_size, pad_type)]).T, 3))
    tmp_im = ImageOps.mirror(tmp_im)
    xc = xc + pad_size
    yc = yc + pad_size
    tmp_im = tmp_im.rotate(angle, center=(xc, yc), resample=Image.BICUBIC)
    
    draw = ImageDraw.Draw(tmp_im) ##################
    w = face_width
    h = w / lambda1 * lambda2
    x1 = anno['y1'] - w/2 + pad_size
    y1 = yc - w / lambda1 * lambda3
    xd = 2*w 
    yd = h
    adj = yd * zoom
    
    crop_x1 = (x1)-((yd-xd)/2)+adj
    crop_y1 = y1+adj
    crop_x2 = (x1)-((yd-xd)/2)+yd-adj
    crop_y2 = y1+yd-adj
     
    rotated_anno = {}
    for i in range(0,5):
        rotated_anno['fm%dx'%i], rotated_anno['fm%dy'%i] = rotate_point((xc, yc), (anno['fm%dx'%i] + pad_size, anno['fm%dy'%i] + pad_size), -angle)

    crop_1 = (crop_x1, crop_y1)
    crop_2 = (crop_x2, crop_y1)
    crop_3 = (crop_x1, crop_y2)
    crop_4 = (crop_x2, crop_y2)
    
    orig_1 = rotate_point((xc, yc), (pad_size, pad_size), -angle)
    orig_2 = rotate_point((xc, yc), (pad_size + im.size[0], pad_size), -angle)
    orig_3 = rotate_point((xc, yc), (pad_size, pad_size + im.size[1]), -angle)
    orig_4 = rotate_point((xc, yc), (pad_size + im.size[0], pad_size + im.size[1]), -angle)

    center = (xc, yc)

    intersect = []
    dist = 999999.0
    nearest_point = None
    for i,j in [(1,2), (2,4), (1,3), (3,4)]:
        for k in (i,j):
            intersect.append((k,eval('line_intersection((orig_%d, orig_%d), (crop_%d, center))'%(i,j,k))))
            d = distance(intersect[-1][-1], center)
            if d < dist:
                dist = d
                nearest_point = intersect[-1]
    
    small_crop = False
    for i in intersect:
        if point_in_rect(i[-1], (crop_x1, crop_y1), (crop_x2, crop_y2)):
            print 'Small Crop'
            small_crop = True
            break
    
    print nearest_point[0]
    if nearest_point[0] == 1:
        x_tmp = crop_2
        y_tmp = crop_3
    if nearest_point[0] == 2:
        x_tmp = crop_1
        y_tmp = crop_4
    if nearest_point[0] == 3:
        x_tmp = crop_4
        y_tmp = crop_1
    if nearest_point[0] == 4:
        x_tmp = crop_3
        y_tmp = crop_2
    
    x2, _ = line_intersection((nearest_point[1], (nearest_point[1][0]+1, nearest_point[1][1])), (x_tmp, center))
    _, y2 = line_intersection((nearest_point[1], (nearest_point[1][0], nearest_point[1][1]+1)), (y_tmp, center))

    
    if not small_crop:
        final_crop_x1 = crop_x1
        final_crop_y1 = crop_y1
        final_crop_x2 = crop_x2
        final_crop_y2 = crop_y2
    else:
        final_crop_x1 = min(nearest_point[1][0], x2)
        final_crop_y1 = min(nearest_point[1][1], y2)
        final_crop_x2 = max(nearest_point[1][0], x2)
        final_crop_y2 = max(nearest_point[1][1], y2)
    for i in range(0,5):
        if not point_in_rect((rotated_anno['fm%dx'%i],rotated_anno['fm%dy'%i]), (final_crop_x1, final_crop_y1), (final_crop_x2, final_crop_y2)):
            print 'Too Close to Boundary'
            break

    crop_pos = []
    x_ratio = 128.0 / (crop_x2 - crop_x1)
    y_ratio = 128.0 / (crop_y2 - crop_y1)
    crop_pos.append([(final_crop_x1 - crop_x1) * x_ratio, (final_crop_y1 - crop_y1) * y_ratio])
    crop_pos.append([(final_crop_x2 - crop_x1) * x_ratio, (final_crop_y2 - crop_y1) * y_ratio])

    orig_pos = []
    orig_pos.append(((orig_1[0] - crop_x1) * x_ratio, (orig_1[1] - crop_y1) * y_ratio))
    orig_pos.append(((orig_2[0] - crop_x1) * x_ratio, (orig_2[1] - crop_y1) * y_ratio))
    orig_pos.append(((orig_4[0] - crop_x1) * x_ratio, (orig_4[1] - crop_y1) * y_ratio))
    orig_pos.append(((orig_3[0] - crop_x1) * x_ratio, (orig_3[1] - crop_y1) * y_ratio))

    if not small_crop:
        return crop_pos, orig_pos, tmp_im.crop((crop_x1,crop_y1,crop_x2,crop_y2)).resize((256,256), resample=Image.BICUBIC)
    else:
        return crop_pos, orig_pos, tmp_im.crop((final_crop_x1,final_crop_y1,final_crop_x2,final_crop_y2)).resize((256,256), resample=Image.BICUBIC)

# def align_eye_pad_ailab_show_alt(im, anno, lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0, zoom=0.0, show=True):
#     if 'fm1x' not in anno.keys():
#         im
#     p1 = np.array((anno['fm1x'], anno['fm1y'])).astype('f')
#     p2 = np.array((anno['fm0x'], anno['fm0y'])).astype('f')
#     face_width = anno['y2'] - anno['y1']
#     angle = angle_between_2_points(p1, p2)
#     x1, y1 = p1
#     x2, y2 = p2
#     xc = (x1 + x2) // 2
#     yc = (y1 + y2) // 2
#     dis_width = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 2.0
#     pad_type = 'constant'
#     pad_size = max(im.size[0], im.size[1]) / 2
#     np_im = np.array(im)
#     tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type, constant_values=255), \
#                                       np.pad(np_im[:,:,1], pad_size, pad_type, constant_values=255), \
#                                       np.pad(np_im[:,:,2], pad_size, pad_type, constant_values=255)]).T, 3))
#     tmp_im = ImageOps.mirror(tmp_im)
#     xc = xc + pad_size
#     yc = yc + pad_size
#     tmp_im = tmp_im.rotate(angle, center=(xc, yc), resample=Image.BICUBIC)
    
#     w = face_width
#     h = w / lambda1 * lambda2
#     x1 = anno['y1'] - w/2 + pad_size
#     y1 = yc - w / lambda1 * lambda3
#     xd = 2*w 
#     yd = h
#     adj = yd * zoom

#     if show:
#         crop_x1 = (x1)-((yd-xd)/2)+adj
#         crop_y1 = y1+adj
#         crop_x2 = (x1)-((yd-xd)/2)+yd-adj
#         crop_y2 = y1+yd-adj
#     else:
#         crop_x1 = x1
#         crop_y1 = y1
#         crop_x2 = x1+xd
#         crop_y2 = y1+yd
    
#     rotated_anno = {}
#     for i in range(0,5):
#         rotated_anno['fm%dx'%i], rotated_anno['fm%dy'%i] = rotate_point((xc, yc), (anno['fm%dx'%i] + pad_size, anno['fm%dy'%i] + pad_size), -angle)

#     crop_1 = (crop_x1, crop_y1)
#     crop_2 = (crop_x2, crop_y1)
#     crop_3 = (crop_x1, crop_y2)
#     crop_4 = (crop_x2, crop_y2)
    
#     orig_1 = rotate_point((xc, yc), (pad_size, pad_size), -angle)
#     orig_2 = rotate_point((xc, yc), (pad_size + im.size[0], pad_size), -angle)
#     orig_3 = rotate_point((xc, yc), (pad_size, pad_size + im.size[1]), -angle)
#     orig_4 = rotate_point((xc, yc), (pad_size + im.size[0], pad_size + im.size[1]), -angle)
    
#     center = (xc, yc)

#     intersect = []
#     dist = 999999.0
#     nearest_point = None
#     for i,j in [(1,2), (2,4), (1,3), (3,4)]:
#         for k in (i,j):
#             intersect.append((k,eval('line_intersection((orig_%d, orig_%d), (crop_%d, center))'%(i,j,k))))
#             d = distance(intersect[-1][-1], center)
#             if d < dist:
#                 dist = d
#                 nearest_point = intersect[-1]
    
#     small_crop = False
#     for i in intersect:
#         if point_in_rect(i[-1], (crop_x1, crop_y1), (crop_x2, crop_y2)):
#             small_crop = True
#             break

#     if nearest_point[0] == 1:
#         x_tmp = crop_2
#         y_tmp = crop_3
#     if nearest_point[0] == 2:
#         x_tmp = crop_1
#         y_tmp = crop_4
#     if nearest_point[0] == 3:
#         x_tmp = crop_4
#         y_tmp = crop_1
#     if nearest_point[0] == 4:
#         x_tmp = crop_3
#         y_tmp = crop_2
    
#     x2, _ = line_intersection((nearest_point[1], (nearest_point[1][0]+1, nearest_point[1][1])), (x_tmp, center))
#     _, y2 = line_intersection((nearest_point[1], (nearest_point[1][0], nearest_point[1][1]+1)), (y_tmp, center))

#     final_crop_x1 = min(nearest_point[1][0], x2)
#     final_crop_y1 = min(nearest_point[1][1], y2)
#     final_crop_x2 = max(nearest_point[1][0], x2)
#     final_crop_y2 = max(nearest_point[1][1], y2)
    
#     for i in range(0,5):
#         if not point_in_rect((rotated_anno['fm%dx'%i],rotated_anno['fm%dy'%i]), (final_crop_x1, final_crop_y1), (final_crop_x2, final_crop_y2)):
# #             print 'Too Close to Boundary'
#             tmp_im = Image.fromarray(np.array(tmp_im) / 4)
#             break
    
#     crop_pos = []
#     x_ratio = 128.0 / (crop_x2 - crop_x1)
#     y_ratio = 128.0 / (crop_y2 - crop_y1)
#     crop_pos.append([(final_crop_x1 - crop_x1) * x_ratio, (final_crop_y1 - crop_y1) * y_ratio])
#     crop_pos.append([(final_crop_x2 - crop_x1) * x_ratio, (final_crop_y2 - crop_y1) * y_ratio])
    
#     orig_pos = []
#     orig_pos.append(((orig_1[0] - crop_x1) * x_ratio, (orig_1[1] - crop_y1) * y_ratio))
#     orig_pos.append(((orig_2[0] - crop_x1) * x_ratio, (orig_2[1] - crop_y1) * y_ratio))
#     orig_pos.append(((orig_4[0] - crop_x1) * x_ratio, (orig_4[1] - crop_y1) * y_ratio))
#     orig_pos.append(((orig_3[0] - crop_x1) * x_ratio, (orig_3[1] - crop_y1) * y_ratio))

#     if not small_crop:
#         print 'sm'
#         return crop_pos, orig_pos, tmp_im.crop((crop_x1,crop_y1,crop_x2,crop_y2)).resize((128,128), resample=Image.BICUBIC)
#     else:
#         return crop_pos, orig_pos, tmp_im.crop((final_crop_x1,final_crop_y1,final_crop_x2,final_crop_y2)).resize((128,128), resample=Image.BICUBIC)

to_show = []
for i in tqdm.tqdm_notebook(range(8)):
    img_path = random.choice(B_path)
    attr = face_detect(img_path)
    crop_pos, _, aligned_img = align_eye_pad_ailab_show_alt(Image.open(img_path), attr, show=True)
    to_show.append(img_pre_128(aligned_img))


show(torchvision.utils.make_grid(to_show),show=False)

In [ ]:
import tqdm

In [1]:
from PIL import Image, ImageDraw

import math

def distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 )

def rotate_point(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point
    radius = np.radians(angle)
    qx = ox + math.cos(radius) * (px - ox) - math.sin(radius) * (py - oy)
    qy = oy + math.sin(radius) * (px - ox) + math.cos(radius) * (py - oy)
    return qx, qy

def draw_point(draw, p, r=20, fill=(255,0,0)):
    if p is None:
        return
    x, y = p
    draw.ellipse(((x-r), (y-r), (x+r), (y+r)), fill=fill)

def angle_between_2_points(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    tan = (y2 - y1) / (x2 - x1)
    return np.degrees(np.arctan(tan))
    
def line_intersection(line1, line2):
    print 'line', line1, line2
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1]) #Typo was here

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y    

def point_in_rect(p, p1, p2):
    return p[0] <= p2[0] and p[0] >= p1[0] and  p[1] <= p2[1] and p[1] >= p1[1]

def align_eye_pad_ailab_show_alt(im, anno, lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0, zoom=0.1):
    p1 = np.array((anno['fm1x'], anno['fm1y'])).astype('f')
    p2 = np.array((anno['fm0x'], anno['fm0y'])).astype('f')
    face_width = anno['y2'] - anno['y1']
    angle = angle_between_2_points(p1, p2)
    x1, y1 = p1
    x2, y2 = p2
    xc = (x1 + x2) // 2
    yc = (y1 + y2) // 2
    dis_width = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 2.0
    pad_type = 'edge'
    pad_size = max(im.size[0], im.size[1]) / 2
    np_im = np.array(im)
    tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type, constant_values=255), \
                                      np.pad(np_im[:,:,1], pad_size, pad_type, constant_values=255), \
                                      np.pad(np_im[:,:,2], pad_size, pad_type, constant_values=255)]).T, 3))
    tmp_im = ImageOps.mirror(tmp_im)
    xc = xc + pad_size
    yc = yc + pad_size
    tmp_im = tmp_im.rotate(angle, center=(xc, yc), resample=Image.BICUBIC)
    
    draw = ImageDraw.Draw(tmp_im) ##################
    w = face_width
    h = w / lambda1 * lambda2
    x1 = anno['y1'] - w/2 + pad_size
    y1 = yc - w / lambda1 * lambda3
    xd = 2*w 
    yd = h
    adj = yd * zoom
#     adj_boundary_error = yd * 0.17

    crop_x1 = (x1)-((yd-xd)/2)+adj
    crop_y1 = y1+adj
    crop_x2 = (x1)-((yd-xd)/2)+yd-adj
    crop_y2 = y1+yd-adj
    print crop_x1, crop_y1, crop_x2, crop_y2
     
    rotated_anno = {}
    for i in range(0,5):
        rotated_anno['fm%dx'%i], rotated_anno['fm%dy'%i] = rotate_point((xc, yc), (anno['fm%dx'%i] + pad_size, anno['fm%dy'%i] + pad_size), -angle)
#     boundary_x1 = (x1)-((yd-xd)/2)+adj_boundary_error
#     boundary_y1 = y1+adj_boundary_error
#     boundary_x2 = (x1)-((yd-xd)/2)+yd-adj_boundary_error
#     boundary_y2 = y1+yd-adj_boundary_error
    
#     draw_point(draw, (boundary_x1, boundary_y1), fill=(255,0,255))
#     draw_point(draw, (boundary_x1, boundary_y2), fill=(255,0,255))
#     draw_point(draw, (boundary_x2, boundary_y2), fill=(255,0,255))
#     draw_point(draw, (boundary_x2, boundary_y1), fill=(255,0,255))
    
    crop_1 = (crop_x1, crop_y1)
    crop_2 = (crop_x2, crop_y1)
    crop_3 = (crop_x1, crop_y2)
    crop_4 = (crop_x2, crop_y2)
    
    draw_point(draw, rotate_point((xc, yc), (pad_size, pad_size), -angle), fill=(0,255,0))
    draw_point(draw, rotate_point((xc, yc), (pad_size + im.size[0], pad_size), -angle), fill=(0,255,0))
    draw_point(draw, rotate_point((xc, yc), (pad_size, pad_size + im.size[1]), -angle), fill=(0,255,0))
    draw_point(draw, rotate_point((xc, yc), (pad_size + im.size[0], pad_size + im.size[1]), -angle), fill=(0,255,0))
    
    orig_1 = rotate_point((xc, yc), (pad_size, pad_size), -angle)
    orig_2 = rotate_point((xc, yc), (pad_size + im.size[0], pad_size), -angle)
    orig_3 = rotate_point((xc, yc), (pad_size, pad_size + im.size[1]), -angle)
    orig_4 = rotate_point((xc, yc), (pad_size + im.size[0], pad_size + im.size[1]), -angle)

    center = (xc, yc)
#     draw_point(draw, line_intersection((orig_1, orig_2), (crop_1, center)), fill=(0,0,255))
#     draw_point(draw, line_intersection((orig_1, orig_2), (crop_2, center)), fill=(0,0,255))
#     draw_point(draw, line_intersection((orig_2, orig_4), (crop_2, center)), fill=(0,0,255))
#     draw_point(draw, line_intersection((orig_2, orig_4), (crop_4, center)), fill=(0,0,255))
#     draw_point(draw, line_intersection((orig_1, orig_3), (crop_3, center)), fill=(0,0,255))
#     draw_point(draw, line_intersection((orig_1, orig_3), (crop_1, center)), fill=(0,0,255))
#     draw_point(draw, line_intersection((orig_3, orig_4), (crop_4, center)), fill=(0,0,255))
#     draw_point(draw, line_intersection((orig_3, orig_4), (crop_3, center)), fill=(0,0,255))
    
    intersect = []
    dist = 999999.0
    nearest_point = None
    for i,j in [(1,2), (2,4), (1,3), (3,4)]:
        for k in (i,j):
            intersect.append((k,eval('line_intersection((orig_%d, orig_%d), (crop_%d, center))'%(i,j,k))))
            print i,j,k,intersect[-1]
            d = distance(intersect[-1][-1], center)
            if d < dist:
                dist = d
                nearest_point = intersect[-1]
    
    small_crop = False
    for i in intersect:
        if point_in_rect(i[-1], (crop_x1, crop_y1), (crop_x2, crop_y2)):
            print 'Small Crop'
            small_crop = True
            break
    
    print nearest_point[0]
    if nearest_point[0] == 1:
        x_tmp = crop_2
        y_tmp = crop_3
    if nearest_point[0] == 2:
        x_tmp = crop_1
        y_tmp = crop_4
    if nearest_point[0] == 3:
        x_tmp = crop_4
        y_tmp = crop_1
    if nearest_point[0] == 4:
        x_tmp = crop_3
        y_tmp = crop_2
    
    x2, _ = line_intersection((nearest_point[1], (nearest_point[1][0]+1, nearest_point[1][1])), (x_tmp, center))
    _, y2 = line_intersection((nearest_point[1], (nearest_point[1][0], nearest_point[1][1]+1)), (y_tmp, center))

    draw_point(draw, nearest_point[1], fill=(255,255,0))
    draw_point(draw, (nearest_point[1][0], y2), fill=(0,255,255))
    draw_point(draw, (x2, y2), fill=(0,255,255))
    draw_point(draw, (x2, nearest_point[1][1]), fill=(0,255,255))
#     draw_point(draw, (max(orig_1[0], crop_1[0]), max(orig_1[1], crop_1[1])), fill=(0,0,255))
#     draw_point(draw, (min(orig_2[0], crop_2[0]), max(orig_1[1], crop_1[1])), fill=(0,0,255))
#     draw_point(draw, (max(orig_1[0], crop_1[0]), min(orig_4[1], crop_4[1])), fill=(0,0,255))
#     draw_point(draw, (min(orig_2[0], crop_2[0]), min(orig_4[1], crop_4[1])), fill=(0,0,255))
    
    draw_point(draw, crop_1)
    draw_point(draw, crop_2)
    draw_point(draw, crop_3)
    draw_point(draw, crop_4)
    
    if not small_crop:
        final_crop_x1 = crop_x1
        final_crop_y1 = crop_y1
        final_crop_x2 = crop_x2
        final_crop_y2 = crop_y2
    else:
        final_crop_x1 = min(nearest_point[1][0], x2)
        final_crop_y1 = min(nearest_point[1][1], y2)
        final_crop_x2 = max(nearest_point[1][0], x2)
        final_crop_y2 = max(nearest_point[1][1], y2)
    for i in range(0,5):
        draw_point(draw, (rotated_anno['fm%dx'%i],rotated_anno['fm%dy'%i]), fill=(255,0,255))
        if not point_in_rect((rotated_anno['fm%dx'%i],rotated_anno['fm%dy'%i]), (final_crop_x1, final_crop_y1), (final_crop_x2, final_crop_y2)):
            print 'Too Close to Boundary'
            break
    del draw #####################################
    
    crop_pos = []
    x_ratio = 128.0 / (crop_x2 - crop_x1)
    y_ratio = 128.0 / (crop_y2 - crop_y1)
    crop_pos.append([(final_crop_x1 - crop_x1) * x_ratio, (final_crop_y1 - crop_y1) * y_ratio])
    crop_pos.append([(final_crop_x2 - crop_x1) * x_ratio, (final_crop_y2 - crop_y1) * y_ratio])
    print x_ratio, y_ratio
    print final_crop_x1, final_crop_y1, final_crop_x2, final_crop_y2
    orig_pos = []
    orig_pos.append(((orig_1[0] - crop_x1) * x_ratio, (orig_1[1] - crop_y1) * y_ratio))
    orig_pos.append(((orig_2[0] - crop_x1) * x_ratio, (orig_2[1] - crop_y1) * y_ratio))
    orig_pos.append(((orig_4[0] - crop_x1) * x_ratio, (orig_4[1] - crop_y1) * y_ratio))
    orig_pos.append(((orig_3[0] - crop_x1) * x_ratio, (orig_3[1] - crop_y1) * y_ratio))
    print crop_pos
    print orig_pos
    print center
    if not small_crop:
        return crop_pos, orig_pos, tmp_im.crop((crop_x1,crop_y1,crop_x2,crop_y2)).resize((128,128), resample=Image.BICUBIC)
    else:
        return crop_pos, orig_pos, tmp_im.crop((final_crop_x1,final_crop_y1,final_crop_x2,final_crop_y2)).resize((128,128), resample=Image.BICUBIC)

img_path = 
attr = face_detect(img_path)
a, b, c = align_eye_pad_ailab_show_alt(Image.open(img_path), attr)
c

NameError: name 'face_detect' is not defined

In [ ]:
Image.open(img_path).size

In [ ]:
img = c
mask = Image.new('RGB', img.size, (255,255,255))
draw = ImageDraw.Draw(mask)
draw.polygon(b, fill=(0,0,0))
del draw
mask

In [ ]:
def align_eye_pad_ailab_full(im, anno, lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0, pad_type='edge'):
    p1 = np.array((anno['fm1x'], anno['fm1y'])).astype('f')
    p2 = np.array((anno['fm0x'], anno['fm0y'])).astype('f')
    face_width = anno['y2'] - anno['y1']
    angle = angle_between_2_points(p1, p2)
    x1, y1 = p1
    x2, y2 = p2
    xc = (x1 + x2) // 2
    yc = (y1 + y2) // 2
    dis_width = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 2.0
    pad_size = max(im.size[0], im.size[1]) / 2
    np_im = np.array(im)
    if pad_type=='constant':
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type, constant_values=255), \
                                          np.pad(np_im[:,:,1], pad_size, pad_type, constant_values=255), \
                                          np.pad(np_im[:,:,2], pad_size, pad_type, constant_values=255)]).T, 3))
    else:
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type ), \
                                  np.pad(np_im[:,:,1], pad_size, pad_type), \
                                  np.pad(np_im[:,:,2], pad_size, pad_type)]).T, 3))
    tmp_im = ImageOps.mirror(tmp_im)
    xc = xc + pad_size
    yc = yc + pad_size
    tmp_im = tmp_im.rotate(angle, center=(xc, yc), resample=Image.BICUBIC)
    w = face_width
    h = w / lambda1 * lambda2
    x1 = anno['y1'] - w/2 + pad_size
    y1 = yc - w / lambda1 * lambda3
    x2 = x1 + 2*w
    y2 = y1 + h
    
    extent = 1.0
    nx1 = x1 - (x2 - x1) * extent / 2
    ny1 = y1 - (y2 - y1) * extent / 2
    nx2 = x2 + (x2 - x1) * extent / 2
    ny2 = y2 + (y2 - y1) * extent / 2
    print nx2 - nx1, x2 - x1
    print ny2 - ny1, y2 - y1
    tmp = 2.0
    return tmp_im.crop((nx1,ny1,nx2,ny2)).resize((int(128*tmp),int(128*tmp)), resample=Image.BICUBIC)

img_show_idt, img_pre_idt = get_idtrans()
# img_path = random.choice(B_path)
# attr = face_detect(img_path)
full_img = align_eye_pad_ailab_full(Image.open(img_path), attr)
_ = show(torchvision.utils.make_grid([img_pre_idt(full_img)]),show=True)
_ = show(torchvision.utils.make_grid([test_img(full_img, net_32['B'], img_pre_idt, 128, bn_eval=False, drop_eval=True).data[0].cpu()]) ,show=True)

In [ ]:
filtered_list = sorted(glob.glob('./filtered/*'))
import tqdm
count_idx = 25
idx = 0

In [ ]:
import shutil
for img_path in glob.glob('../fake_detect/haozhi_val/*'):
    if rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1:
        continue
    shutil.copy2(img_path, os.path.join('./to_filter'))

In [ ]:
len(test_set)

In [ ]:
B_path = glob.glob('../fake_detect/haozhi_val/*')
# B_path = glob.glob('./datasets/getchu_aisa_aligned_cleaned_v1/testB/*')
# my_path = glob.glob('../mytestimg/*/*')
img_path =  random.choice(B_path)
# img_path = '/data2/minjunli/inbox/testimg/nan8.jpg'
# img_path = '/data2/minjunli/inbox/testimg/nan1.jpg'
# nan_path = glob.glob('../face_lib_points/testimg/nan*')
# img_path = random.choice(nan_path)
# while rev_attr[img_path.split('/')[-1]]['machanno:FaceFullAnnotator']['data'][0]['gender'] == 1:
#     img_path = random.choice(B_path)
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = align(img, attr)
img_show = align_show(img, attr)
# img_aligned = img
img_masked = predict_mask_alt(img_aligned)
test_net = range(37)

dropout = set([7,4,8,9,13,15,12,16,24,25,26,35,32,28,29,18,31,36,23])

for i in test_net:
    exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

blank_img = Image.new('RGB',(128,128),(255,255,255))
to_show = [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show)]
for i in range(7):
    to_show.append(img_pre_128(blank_img))

for i in test_net:
    if i not in dropout:
        to_show.append(eval('post_process(fake_A_%d.data[0].cpu(),col_add=8)'%i))
    else:
        to_show.append(eval('post_process(fake_A_%d.data[0].cpu()) / 3'%i))
    
show(torchvision.utils.make_grid(to_show,nrow=10),show=True)



In [ ]:
show(post_process(fake_A_2.data[0].cpu(),crop_pos=None, col_add=8),show=False)#.save('./report_show/tmp9.png')

In [ ]:
fake_A_2.data[0].cpu().shape

def narrow_cut(t, col=1):
    tmp = torch.ones(3, 128, col)
    to_show_pre = transforms.Compose([transforms.Scale((128,128), Image.BICUBIC)
                                      ,transforms.ToTensor()
                                      ,transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
    return to_show_pre(Image.fromarray(util.tensor2im(torch.cat((tmp, t, tmp), 2).view(1,3,128,-1))))

def narrow(t, col=1):
    tmp = torch.ones(3, 128, col)
    to_show_pre = transforms.Compose([transforms.Scale((128,128), Image.BICUBIC)
                                      ,transforms.ToTensor()
                                      ,transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
    return to_show_pre(Image.fromarray(util.tensor2im(torch.cat((tmp, t, tmp), 2).view(1,3,128,-1))))

show(narrow(fake_A_2.data[0].cpu(),10),show=False)
# show(pre(narrow(fake_A_2.data[0].cpu(),10)))


In [ ]:
%%time
# B_path = glob.glob('../fake_detect/haozhi_val/*')
# my_path = glob.glob('../mytestimg/*/*')
# img_path = '/data2/minjunli/inbox/testimg/nan12.jpg'
img_path = random.choice(B_path)
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = align(img, attr)
img_show = align_show(img, attr)
# img_aligned = img
img_masked = predict_mask_alt(img_aligned)

dropout = set([8,9,13,26,16,28,29,5,35,12,19,18,7,25,10])

to_show = [img_pre_128(img), img_pre_128(img_masked), img_pre_128(img_show)]
for i in range(7):
    to_show.append(img_pre_128(blank_img))

# img_path = './datasets/getchu_aisacelebaselfie_aligned_cleaned_masked_blur_v6/trainB/'+img_path.split('/')[-1]
# attr = face_detect(img_path)
#img_aligned = align(img, attr)

for i in range(37,44):
    exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

# tmp = show(torchvision.utils.make_grid([img_pre_128(img), img_pre_128(img_aligned), fake_A_1.data[0].cpu(), fake_A_2.data[0].cpu(), fake_A_3.data[0].cpu(), fake_A_4.data[0].cpu(), fake_A_5.data[0].cpu(), fake_A_6.data[0].cpu(), fake_A_7.data[0].cpu(), fake_A_8.data[0].cpu()]),show=True)
# to_show = [img_pre_128(img), img_pre_128(img_aligned)]
for i in range(37,44):
    if i not in dropout:
        to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i))
    else:
        to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8) / 3'%i))

_ = show(torchvision.utils.make_grid(to_show,nrow=10),show=True)

In [ ]:
%%time
B_path = glob.glob('../fake_detect/haozhi_val/*')
# my_path = glob.glob('../mytestimg/*/*')
# img_path = '/data2/minjunli/inbox/testimg/nan12.jpg'
# img_path = random.choice(B_path)
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = align(img, attr)
img_show = align_show(img, attr)
# img_aligned = img
img_masked = predict_mask_alt(img_aligned)

dropout = set([8,9,13,26,16,28,29,5,35,12,19,18,7,25,10])

to_show = [img_pre_128(img), img_pre_128(img_masked), img_pre_128(img_show)]
for i in range(7):
    to_show.append(img_pre_128(blank_img))

# img_path = './datasets/getchu_aisacelebaselfie_aligned_cleaned_masked_blur_v6/trainB/'+img_path.split('/')[-1]
# attr = face_detect(img_path)
#img_aligned = align(img, attr)

for i in test_net:
    exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

# tmp = show(torchvision.utils.make_grid([img_pre_128(img), img_pre_128(img_aligned), fake_A_1.data[0].cpu(), fake_A_2.data[0].cpu(), fake_A_3.data[0].cpu(), fake_A_4.data[0].cpu(), fake_A_5.data[0].cpu(), fake_A_6.data[0].cpu(), fake_A_7.data[0].cpu(), fake_A_8.data[0].cpu()]),show=True)
# to_show = [img_pre_128(img), img_pre_128(img_aligned)]
for i in test_net:
    if i not in dropout:
        to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i))
    else:
        to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8) / 3'%i))

_ = show(torchvision.utils.make_grid(to_show,nrow=10),show=True)

In [ ]:
# B_path = glob.glob('./datasets/getchu_aisa_aligned_cleaned_v1/trainB/*')
# my_path = glob.glob('../mytestimg/seiyuu/*')
# img_path = random.choice(B_path)
# nan_path = glob.glob('../face_lib_points/testimg/nan*')
# img_path = random.choice(nan_path)
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')

img_aligned = align(img, attr)
img_show = align_show(img, attr)
# img_aligned = img
img_masked = predict_mask_alt(img_aligned)

test_net = [33,28,37,38,39,40,41,42,43]

dropout = set([4,8,9,15,16,24,25,26])

for i in test_net:
    exec('fake_A_%d = test_img(img_aligned, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

blank_img = Image.new('RGB',(128,128),(255,255,255))
to_show = [img_pre_128(img), img_pre_128(img_aligned), img_pre_128(img_show)]
for i in range(6):
    to_show.append(img_pre_128(blank_img))

for i in test_net:
    to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i))
        
for i in test_net:
    exec('fake_A_%d = test_img(img_masked, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

for i in test_net:
    to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i))
    
for i in test_net:
    exec('fake_A_%d = test_img(img_show, net_%d[\'B\'], img_pre_128, 128, bn_eval=False, drop_eval=True)'%(i,i))

for i in test_net:
    to_show.append(eval('narrow(fake_A_%d.data[0].cpu(),8)'%i))
    
    
_ = show(torchvision.utils.make_grid(to_show,nrow=9),show=True)



In [ ]:
from PIL import ImageFilter
def get_mask_alt(img):
    image_width =  None
    image_height = None

    image_base64 = _to_img(img, image_width, image_height)

    r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})

    #print r.content
    js = r.json()
    #print js
    mask = _get_img(js['prob'], image_width, image_height)
    #image_matting = _get_img(js['img_data'], image_width, image_height)
    img_npy = np.asarray(img)
    white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255
    softmask = (1-np.array(mask.filter(ImageFilter.GaussianBlur))/255.0)

    return softmask

def predict_mask_alt(img):
    image_width =  None
    image_height = None
    print "mask", img.size
    image_base64 = _to_img(img, image_width, image_height)
    
    r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})

    #print r.content
    js = r.json()
    #print js
    mask = _get_img(js['prob'], image_width, image_height)
    #image_matting = _get_img(js['img_data'], image_width, image_height)
    img_npy = np.asarray(img)
    white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255
    softmask = (1-np.array(mask.filter(ImageFilter.GaussianBlur))/255.0)
    masked_img = img * (1-softmask) + white_img_npy * (softmask)

    return Image.fromarray(masked_img.astype(np.uint8))

def predict_mask(img):
    image_width =  None
    image_height = None

    image_base64 = _to_img(img, image_width, image_height)

    r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})
    print r.status_code
    #print r.content
    js = r.json()
    #print js
    mask = _get_img(js['prob'], image_width, image_height)
    #image_matting = _get_img(js['img_data'], image_width, image_height)
    img_npy = np.asarray(img)
    white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255
    hardmask = (1-np.array(mask)/255.0)
    masked_img = img * (1-hardmask) + white_img_npy * (hardmask)

    return Image.fromarray(masked_img.astype(np.uint8))

In [ ]:
import tqdm
B_path = glob.glob('./datasets/getchu_aisacelebaselfie_aligned_v6/trainB/*')
tmp_ = None
for i in tqdm.tqdm_notebook(range(1000)):
    p = random.random()
    img_path = B_path[i]
#     if p > 0.5:
#         img_path = './datasets/getchu_aisacelebaselfie_aligned_cleaned_masked_blur_v6/trainB/'+img_path.split('/')[-1]
    img = Image.open(img_path).convert('RGB')
    img_aligned = img
    #fake_A_7 = test_img(img_aligned, net_15['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
    fake_A_8 = test_img(img_aligned, net_6['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)

    show(fake_A_8.data[0].cpu(),show=False).save('../fake_detect/gliches/'+img_path.split('/')[-1])



In [ ]:
img_path = '../face_align/test/masktest.png'
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = align_eye_pad_ailab(img, attr, pad_type='edge')
crop_pos, orig_pos, img_show = align_eye_pad_ailab_show_alt(img, attr, zoom=0.1, show=True)
#img_aligned = img_show
crop_pos = [[0,0],[128,128]]
print crop_pos
img_masked = predict_mask_alt(img_aligned)

In [ ]:
img_masked.resize((128,128))

In [ ]:
Image.open('../face_align/test/masktest.png.output.png')

In [ ]:
from PIL import ImageFilter

def predict_mask_alt(img):
    image_width =  None
    image_height = None

    image_base64 = _to_img(img, image_width, image_height)

    r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})
    #print r.status_code
    #print r.content
    js = r.json()
    #print js
    mask = _get_img(js['prob'], image_width, image_height)
    #image_matting = _get_img(js['img_data'], image_width, image_height)
    img_npy = np.asarray(img)
    white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255
    softmask = (1-np.array(mask.filter(ImageFilter.GaussianBlur))/255.0)
    masked_img = img * (1-softmask) + white_img_npy * (softmask)

    return Image.fromarray(masked_img.astype(np.uint8))

B_path = glob.glob('../fake_detect/aligned/*')
tmp_ = None
# for i in tqdm.tqdm_notebook(range(len(B_path))):
p = random.random()
img_path = random.choice(B_path)
# if 'json' in img_path:
#     continue
#     attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
#     if 'fm1x' not in attr.keys():
#         continue
img_aligned = img
if p > 0.5:
    img_aligned = predict_mask_alt(img_aligned)
#fake_A_7 = test_img(img_aligned, net_15['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
fake_A_8 = test_img(img_aligned, net_16['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
show(fake_A_8.data[0].cpu(),show=False)


In [ ]:
Image.open(glob.glob('../fake_detect/aligned/*')[0])

In [ ]:
B_path = glob.glob('../fake_detect/aligned/*')
tmp_ = None
for i in tqdm.tqdm_notebook(range(len(B_path))):
    p = random.random()
    img_path = B_path[i]
    if 'json' in img_path:
        continue
    attr = face_detect(img_path)
    img = Image.open(img_path).convert('RGB')
    img_aligned = align(img, attr)
    img_masked = predict_mask_alt(img_aligned)
    #fake_A_7 = test_img(img_aligned, net_15['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
    fake_A = test_img(img_aligned, net_16['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
    show(fake_A_8.data[0].cpu(),show=False).save('../fake_detect/result_filtered/'+img_path.split('/')[-1])


In [ ]:
B_path = glob.glob('../fake_detect/random/*')
tmp_ = None
for i in tqdm.tqdm_notebook(range(len(B_path))):
    p = random.random()
    img_path = B_path[i]
    if 'json' in img_path:
        continue
    attr = face_detect(img_path)
    img = Image.open(img_path).convert('RGB')
    img_aligned = align(img, attr)
    if p > 0.5:
        img_aligned = predict_mask_alt(img_aligned)
    #fake_A_7 = test_img(img_aligned, net_15['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
    fake_A_8 = test_img(img_aligned, net_16['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
    show(fake_A_8.data[0].cpu(),show=False).save('../fake_detect/result_random/'+img_path.split('/')[-1])


In [ ]:
tmp1 = align(Image.open('../face_lib_points/test/nan2.jpg'), attr = face_detect('../face_lib_points/test/nan2.jpg'))
tmp2 = Image.open('../face_lib_points/test/nan2.jpg.output.jpg')

fake_A_1 = test_img(tmp1, net_15['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
fake_A_2 = test_img(tmp1, net_2['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
fake_A_3 = test_img(tmp2, net_15['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
fake_A_4 = test_img(tmp2, net_2['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
tmp = show(torchvision.utils.make_grid([img_pre_128(tmp1), fake_A_1.data[0].cpu(), fake_A_2.data[0].cpu(), 
                                        img_pre_128(tmp2), fake_A_3.data[0].cpu(), fake_A_4.data[0].cpu()], nrow=3),show=True)

In [ ]:
tmp1 = align(Image.open('../face_lib_points/test/nan2.jpg'), attr = face_detect('../face_lib_points/test/nan2.jpg'))
tmp2 = Image.open('../face_lib_points/test/nan2.jpg.output.jpg')
tmpshow = align_eye_pad_ailab_show(Image.open('../face_lib_points/test/nan2.jpg'), anno=face_detect('../face_lib_points/test/nan2.jpg'), lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0)

fake_A_1 = test_img(tmp1, net_11['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
fake_A_2 = test_img(tmp1, net_12['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
fake_A_3 = test_img(tmp2, net_11['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
fake_A_4 = test_img(tmp2, net_12['B'], img_pre_128, 128, bn_eval=False, drop_eval=True)
tmp = show(torchvision.utils.make_grid([img_pre_128(tmpshow), fake_A_1.data[0].cpu(), fake_A_2.data[0].cpu(), 
                                        img_pre_128(tmpshow), fake_A_3.data[0].cpu(), fake_A_4.data[0].cpu()], nrow=3),show=True)

In [ ]:
def align_eye_pad_ailab_show(im, anno, lambda1, lambda2, lambda3):
    p1 = np.array((anno['fm1x'], anno['fm1y'])).astype('f')
    p2 = np.array((anno['fm0x'], anno['fm0y'])).astype('f')
    face_width = anno['y2'] - anno['y1']
    angle = angle_between_2_points(p1, p2)
    x1, y1 = p1
    x2, y2 = p2
    xc = (x1 + x2) // 2
    yc = (y1 + y2) // 2
    dis_width = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 2.0
    pad_type = 'edge'
    pad_size = max(im.size[0], im.size[1]) / 2
    np_im = np.array(im)
    tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type), \
                                      np.pad(np_im[:,:,1], pad_size, pad_type), \
                                      np.pad(np_im[:,:,2], pad_size, pad_type)]).T, 3))
    tmp_im = ImageOps.mirror(tmp_im)
    xc = xc + pad_size
    yc = yc + pad_size
    tmp_im = tmp_im.rotate(angle, center=(xc, yc), resample=Image.BICUBIC)
    w = face_width
    h = w / lambda1 * lambda2
    x1 = anno['y1'] - w/2 + pad_size
    y1 = yc - w / lambda1 * lambda3
    xd = 2*w
    yd = h
    print xd, yd
    return tmp_im.crop((x1-(yd-xd)/2,y1,x1-(yd-xd)/2+yd,y1+yd)).resize((128,128), resample=Image.BICUBIC)

tmp = align_eye_pad_ailab_show(Image.open('../face_lib_points/test/nan2.jpg'), anno=face_detect('../face_lib_points/test/nan2.jpg'), lambda1 = 77.0, lambda2 = 228.0, lambda3 = 111.0)

In [ ]:
tmp

In [ ]:
Image.open('../face_lib_points/test/nan2.jpg')

In [ ]:
# m = net_12['B']
def set_eval(net, bn=True, drop=True):
    if isinstance(net, torch.nn.Dropout):
        net.training = not drop
    elif  isinstance(net, torch.nn.BatchNorm2d) or  isinstance(net, torch.nn.InstanceNorm2d):
        net.training = not bn
    else:
        net.training = False
    for i in net.children():
        set_eval(i, bn, drop)

In [ ]:
net_1['B']

In [ ]:
img_masked = predict_mask(img_aligned)
fake_A_3 = test_img(img_masked, net_3['B'], img_pre_128, 128, eval_mode=True)
fake_A_4 = test_img(img_masked, net_4['B'], img_pre_128, 128, eval_mode=True)
fake_A_7 = test_img(img_masked, net_7['B'], img_pre_128, 128, eval_mode=True)
fake_A_8 = test_img(img_masked, net_8['B'], img_pre_128, 128, eval_mode=True)
fake_A_9 = test_img(img_masked, net_9['B'], img_pre_128, 128, eval_mode=True)
show(torchvision.utils.make_grid([img_pre_128(img_aligned), img_pre_128(img_masked), fake_A_3.data[0].cpu(), fake_A_4.data[0].cpu(), fake_A_7.data[0].cpu(), fake_A_8.data[0].cpu(), fake_A_9.data[0].cpu()]))

In [ ]:
B_p = random.choice(B_path)
name = B_p.split('/')[-1]
# B_p = os.path.join('../../../clean_dataset/out/', B_p)
B_ = Image.open(B_p).convert('RGB').resize((128,128), Image.LANCZOS)#.crop((2,2,126,126))

fake_A_1 = test_img(B_, net_1['B'], img_pre_128, 128,eval_mode=True)
fake_A_2 = test_img(B_, net_2['B'], img_pre_128, 128, eval_mode=True)
fake_A_3 = test_img(B_, net_3['B'], img_pre_128, 128, eval_mode=True)
fake_A_4 = test_img(B_, net_4['B'], img_pre_128, 128, eval_mode=True)
#fake_A_5 = test_img(B_, net_5['B'], img_pre_128, 128, eval_mode=True)
#fake_A_6 = test_img(B_, net_6['B'], img_pre_128, 128, eval_mode=True)
fake_A_7 = test_img(B_, net_7['B'], img_pre_128, 128, eval_mode=True)
fake_A_8 = test_img(B_, net_8['B'], img_pre_128, 128, eval_mode=True)
fake_A_9 = test_img(B_, net_7['B'], img_pre_128, 128, eval_mode=True)
fake_A_10 = test_img(B_, net_8['B'], img_pre_128, 128, eval_mode=True)
#show(torchvision.utils.make_grid([img_pre_128(B_),fake_A_1.data[0].cpu(),fake_A_2.data[0].cpu(),fake_A_3.data[0].cpu(),fake_A_4.data[0].cpu(),fake_A_5.data[0].cpu(),fake_A_6.data[0].cpu()]),show=False)
show(torchvision.utils.make_grid([img_pre_128(B_),fake_A_1.data[0].cpu(),fake_A_2.data[0].cpu(),fake_A_3.data[0].cpu(),fake_A_4.data[0].cpu(),fake_A_7.data[0].cpu(),fake_A_8.data[0].cpu(),fake_A_9.data[0].cpu()]),show=False)

In [ ]:
#B_p = random.choice(B_path)
#name = B_p.split('/')[-1]
# B_p = os.path.join('../../../clean_dataset/out/', B_p)


def predict_mask(img):
    image_width =  None
    image_height = None

    image_base64 = _to_img(img, image_width, image_height)

    r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})
    print r.status_code
    #print r.content
    js = r.json()
    #print js
    mask = _get_img(js['prob'], image_width, image_height)
    #image_matting = _get_img(js['img_data'], image_width, image_height)

    img_npy = np.asarray(img)

    mask = mask.point(lambda p: p > 50 and 255) 
    #mask = mask.point(lambda p: p < 128 or 0)

    mask_npy = np.asarray(mask) / 255

    mask_inv_npy = 1 - mask_npy
    white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255


    img_masked_npy = np.multiply(img_npy, mask_npy) + np.multiply(white_img_npy, mask_inv_npy)
    img_masked = Image.fromarray(img_masked_npy)
    return img_masked

img = Image.open(B_p).convert('RGB').resize((128,128), Image.LANCZOS)#.crop((2,2,126,126))
img_masked = predict_mask(img)
fake_A_1 = test_img(img_masked, net_1['B'], img_pre_128, 128, eval_mode=True)
fake_A_2 = test_img(img_masked, net_2['B'], img_pre_128, 128, eval_mode=True)
fake_A_3 = test_img(img_masked, net_3['B'], img_pre_128, 128, eval_mode=True)
fake_A_4 = test_img(img_masked, net_4['B'], img_pre_128, 128, eval_mode=True)
#fake_A_5 = test_img(img_masked, net_5['B'], img_pre_128, 128, eval_mode=True)
#fake_A_6 = test_img(img_masked, net_6['B'], img_pre_128, 128, eval_mode=True)
fake_A_7 = test_img(img_masked, net_7['B'], img_pre_128, 128, eval_mode=True)
fake_A_8 = test_img(img_masked, net_8['B'], img_pre_128, 128, eval_mode=True)
fake_A_9 = test_img(img_masked, net_9['B'], img_pre_128, 128, eval_mode=True)
#show(torchvision.utils.make_grid([img_pre_128(img_masked),fake_A_1.data[0].cpu(),fake_A_2.data[0].cpu(),fake_A_3.data[0].cpu(),fake_A_4.data[0].cpu(), fake_A_5.data[0].cpu(),fake_A_6.data[0].cpu()]),show=False)
show(torchvision.utils.make_grid([img_pre_128(img_masked),fake_A_1.data[0].cpu(),fake_A_2.data[0].cpu(),fake_A_3.data[0].cpu(),fake_A_4.data[0].cpu(), fake_A_7.data[0].cpu(),fake_A_8.data[0].cpu(), fake_A_9.data[0].cpu()]),show=False)

In [ ]:
import cv2

def hisEqulColor(img):
    img_np = np.array(img)
    img_cv = img_np[:,:,::-1]
    ycrcb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2YCR_CB)
    channels = cv2.split(ycrcb)
    cv2.equalizeHist(channels[0], channels[0])
    cv2.merge(channels, ycrcb)
    img_cv = cv2.cvtColor(ycrcb, cv2.COLOR_YCR_CB2BGR)
    img_np = img_cv[:,:,::-1]
    img = Image.fromarray(img_np)
    return img

B_ = Image.open(B_p).convert('RGB').resize((128,128), Image.LANCZOS)#.crop((2,2,126,126))

B_ = hisEqulColor(B_)

fake_A_1 = test_img(B_, net_1['B'], img_pre_128, 128,eval_mode=True)
fake_A_2 = test_img(B_, net_2['B'], img_pre_128, 128, eval_mode=True)
fake_A_3 = test_img(B_, net_3['B'], img_pre_128, 128, eval_mode=True)
fake_A_4 = test_img(B_, net_4['B'], img_pre_128, 128, eval_mode=True)
#fake_A_5 = test_img(B_, net_5['B'], img_pre_128, 128, eval_mode=True)
#fake_A_6 = test_img(B_, net_6['B'], img_pre_128, 128, eval_mode=True)
fake_A_7 = test_img(B_, net_7['B'], img_pre_128, 128, eval_mode=True)
fake_A_8 = test_img(B_, net_8['B'], img_pre_128, 128, eval_mode=True)
fake_A_9 = test_img(B_, net_7['B'], img_pre_128, 128, eval_mode=True)
fake_A_10 = test_img(B_, net_8['B'], img_pre_128, 128, eval_mode=True)
show(torchvision.utils.make_grid([img_pre_128(B_), fake_A_1.data[0].cpu(),fake_A_2.data[0].cpu(),fake_A_3.data[0].cpu(),fake_A_4.data[0].cpu(),fake_A_7.data[0].cpu(),fake_A_8.data[0].cpu(),fake_A_9.data[0].cpu()]),show=False)







In [ ]:
import cv2

def hisEqulColor(img):
    img_np = np.array(img)
    img_cv = img_np[:,:,::-1]
    ycrcb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2YCR_CB)
    channels = cv2.split(ycrcb)
    cv2.equalizeHist(channels[0], channels[0])
    cv2.merge(channels, ycrcb)
    img_cv = cv2.cvtColor(ycrcb, cv2.COLOR_YCR_CB2BGR)
    img_np = img_cv[:,:,::-1]
    img = Image.fromarray(img_np)
    return img

B_ = Image.open(B_p).convert('RGB').resize((128,128), Image.LANCZOS)#.crop((2,2,126,126))

img = hisEqulColor(B_)

image_width =  None
image_height = None

image_base64 = _to_img(img, image_width, image_height)

r = requests.post(URL, json={"session_id": "xiaolongzhu", "img_data": image_base64})
print r.status_code
#print r.content
js = r.json()
#print js
mask = _get_img(js['prob'], image_width, image_height)
#image_matting = _get_img(js['img_data'], image_width, image_height)

img_npy = np.asarray(img)

mask = mask.point(lambda p: p > 50 and 255) 
#mask = mask.point(lambda p: p < 128 or 0)

mask_npy = np.asarray(mask) / 255

mask_inv_npy = 1 - mask_npy
white_img_npy = np.ones((img_npy.shape),dtype=np.uint8) * 255


img_masked_npy = np.multiply(img_npy, mask_npy) + np.multiply(white_img_npy, mask_inv_npy)
img_masked = Image.fromarray(img_masked_npy)


fake_A_1 = test_img(img_masked, net_1['B'], img_pre_128, 128, eval_mode=True)
fake_A_2 = test_img(img_masked, net_2['B'], img_pre_128, 128, eval_mode=True)
fake_A_3 = test_img(img_masked, net_3['B'], img_pre_128, 128, eval_mode=True)
fake_A_4 = test_img(img_masked, net_4['B'], img_pre_128, 128, eval_mode=True)
#fake_A_5 = test_img(img_masked, net_5['B'], img_pre_128, 128, eval_mode=True)
#fake_A_6 = test_img(img_masked, net_6['B'], img_pre_128, 128, eval_mode=True)
fake_A_7 = test_img(img_masked, net_7['B'], img_pre_128, 128, eval_mode=True)
fake_A_8 = test_img(img_masked, net_8['B'], img_pre_128, 128, eval_mode=True)
fake_A_9 = test_img(img_masked, net_9['B'], img_pre_128, 128, eval_mode=True)
#show(torchvision.utils.make_grid([img_pre_128(img_masked),fake_A_1.data[0].cpu(),fake_A_2.data[0].cpu(),fake_A_3.data[0].cpu(),fake_A_4.data[0].cpu(), fake_A_5.data[0].cpu(),fake_A_6.data[0].cpu()]),show=False)
show(torchvision.utils.make_grid([img_pre_128(img_masked),fake_A_1.data[0].cpu(),fake_A_2.data[0].cpu(),fake_A_3.data[0].cpu(),fake_A_4.data[0].cpu(), fake_A_7.data[0].cpu(),fake_A_8.data[0].cpu(), fake_A_9.data[0].cpu()]),show=False)




In [ ]:
B_path

In [ ]:
from IPython.html.widgets import interact
from PIL import Image

In [ ]:
from PIL import ImageOps

def resize_plot(x=0.37, d=0.87, rot=1.0):
    B_ = Image.open(B_p).convert('RGB')#.crop((1445,0,3666,2028))
    max_base_width = 512
    wpercent = (max_base_width/float(B_.size[0]))
    hsize = int((float(B_.size[1])*float(wpercent)))
    B_1 = B_.resize((max_base_width,hsize), Image.BICUBIC).rotate(0)
    B_2 = B_1
    if rot!= 0:
        B_2_orig = B_2
        np_im = np.array(B_2)
        pad_type = 'edge'
        pad_size = max(B_2.size[0], B_2.size[1]) / 2
        tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type), \
                                  np.pad(np_im[:,:,1], pad_size, pad_type), \
                                  np.pad(np_im[:,:,2], pad_size, pad_type)]).T, 3))
        B_2 = ImageOps.mirror(tmp_im)
        B_2 = B_2.rotate(rot, Image.BICUBIC)
        B_2 = B_2.crop((pad_size, pad_size, pad_size+B_2_orig.size[0], pad_size+B_2_orig.size[1]))
    B_2 = B_2.resize((int((B_1.size[0])*x),int((B_1.size[1])*(x*d))), Image.BICUBIC)
#     B_2_pad = Image.new('RGB',(B_2.size[0]+60, B_2.size[1]+60), (0,0,0))
#     np_im = np.array(B_2)
#     pad_type = 'mean'
#     pad_size = 30
#     tmp_im = Image.fromarray(np.rot90(np.array([np.pad(np_im[:,:,0], pad_size, pad_type), \
#                               np.pad(np_im[:,:,1], pad_size, pad_type), \
#                               np.pad(np_im[:,:,2], pad_size, pad_type)]).T, 3))
#     B_2_pad = ImageOps.mirror(tmp_im)
#     B_2 = B_2_pad
#     print B_2_pad.size, B_2.size
    fake_A_1 = test_img(B_2, net_2['B'], img_id_pre, 128, eval_mode=True)
    B_2_show = Image.new('RGB', (fake_A_1.shape[-1], fake_A_1.shape[-2]), (0,0,0))
    print B_2.size, B_2_show.size
    B_2_show.paste(B_2, (0,0))#B_2.getbbox())
    show(torchvision.utils.make_grid([img_id_pre(B_2_show),fake_A_1.data[0].cpu()]))

interact(resize_plot, x=(0,1,0.01), d=(0.5,1.2,0.01), rot=(-30,30,1))

In [ ]:
def resize_plot2(x=0.27, d=0.0):
    B_ = Image.open(B_p).convert('RGB').crop((129,0,479,464))
    base_width = 512
    wpercent = (base_width/float(B_.size[0]))
    hsize = int((float(B_.size[1])*float(wpercent)))
    B_1 = B_.resize((base_width,hsize), Image.BICUBIC)
    B_2 = B_1.resize((int((B_1.size[0])*x),int((B_1.size[1])*(x+d))), Image.ANTIALIAS)
    fake_A_1 = test_img(B_2, net_2['B'], img_id_pre, 128, eval_mode=True)
    B_2_show = Image.new('RGB', (fake_A_1.shape[-1], fake_A_1.shape[-2]), (0,0,0))
    print B_2.size, B_2_show.size
    B_2_show.paste(B_2, B_2.getbbox())
    #print net_1_d['B'].forward(fake_A_1).mean().data 
    try:
        show(torchvision.utils.make_grid([img_id_pre(B_2_show),fake_A_1.data[0].cpu()]))
    except Exception:
        show(torchvision.utils.make_grid([img_id_pre(B_2_show)]))
        show(torchvision.utils.make_grid([fake_A_1.data[0].cpu()]))

interact(resize_plot2, x=(0,1,0.01), d=(-0.5,0.5,0.01))

In [ ]:
net_1 = get_nets('./checkpoints/Get_AisCel_128_4_aff_1_resize_caffe/')

In [ ]:

img_path = '/data2/minjunli/prj/anime/face_lib_points/test/testimg/test.png'
attr = face_detect(img_path)
img = Image.open(img_path).convert('RGB')
img_aligned = align(img, attr)

fake_A_1 = test_img(img_aligned, net_1['B'], img_pre_128, 128, eval_mode=True)
show(torchvision.utils.make_grid([img_pre_128(img), img_pre_128(img_aligned), fake_A_1.data[0].cpu()]),show=False)


In [ ]:
%pdb
import pdb
import torch
from torch.autograd import Variable
import torchvision

import os
import pytorch2caffe

m = net_1['B'].cpu()
m.eval()
print(m)

input_var = Variable(torch.rand(1, 3, 128, 128))
output_var = m(input_var)

output_dir = 'caffe'
pytorch2caffe.pytorch2caffe(input_var, output_var, 
              os.path.join(output_dir, 'trans.prototxt'),
              os.path.join(output_dir, 'trans.caffemodel'))

In [ ]:
reload(pytorch2caffe)

In [ ]:
import sys
sys.path.append("/data2/minjunli/tol/caffe/python/")
sys.path.append("/data2/minjunli/tol/pytorch2caffe/")

In [ ]:
%pdb
import pdb
pdb.set_trace()

In [ ]:
import caffe

In [ ]:
net = caffe.Net('./caffe/trans.prototxt', caffe.TEST)